<a href="https://colab.research.google.com/github/davidfague/Stylized-ReducedOrder-L5-Model/blob/main/SROL5_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
server=False # Change for whether or not you are SSHing into the server
github=True # Change for whether or not you want to get the model from github or google drive

#### Neuron Initialization
Import modules and setup simulation parameters

In [2]:
import shutil
if server==True:
  print('Model should already be in directory.') #supports large data files in directory
  tstop=1000
elif github==True: # If you do not already have the model you can clone from Github; large data files are not stored on Github
  try: shutil.rmtree('Stylized-ReducedOrder-L5-Model/') #removes folder if already cloned #uncomment to pull the latest from github if pushing during notebook use.
  except:
    print('Stylized-ReducedOrder-L5-Model/ not in directory')
  !git clone https://github.com/davidfague/Stylized-ReducedOrder-L5-Model.git
  %cd Stylized-ReducedOrder-L5-Model/
else: #If you have the folder in your google drive (easier to update geom_parameters), and allows uploading of large data files
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Stylized-ReducedOrder-L5-Model/
  tstop=1000

Cloning into 'Stylized-ReducedOrder-L5-Model'...
remote: Enumerating objects: 1605, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 1605 (delta 178), reused 180 (delta 120), pack-reused 1307
Receiving objects: 100% (1605/1605), 113.05 MiB | 18.07 MiB/s, done.
Resolving deltas: 100% (810/810), done.
/content/Stylized-ReducedOrder-L5-Model


In [3]:
!pip install neuron
from neuron import h,nrn
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import time as time
from io import StringIO
import os, sys
from typing import Optional, List, Tuple, Union
from functools import partial
import scipy.signal as ss
import scipy.stats as st
import h5py
import matplotlib.pyplot as plt
import math
import random
random.seed(427)
from utils.plot_results import plot_LFP_traces,plot_LFP_heatmap
from utils.plot_morphology import plot_morphology
import pdb #python debugger
# from scipy.stats.distributions import TRUNCNORM_MAX_BRENT_ITERS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from stylized_module.stylized_cell import Stylized_Cell
from stylized_module.ecp import EcpMod, newposition
from stylized_module.recorder import Recorder
from stylized_module.point_current import Point_current
from SROL5.utils import *
# from SROL5.utils import Record_Axial_Current

mechanisms_path = 'mechanisms'  # need to compile the folder first
geometry_file = 'geom_parameters.csv'
try:
    tstop = tstop  # ms simulation duration
except:
    tstop = 1000 #if tstop is not defined previously
dt = 0.1  # ms. does not allow change

h.load_file('stdrun.hoc')
#h.nrn_load_dll(os.path.join(mechanisms_path,'nrnmech.dll'))  # running this line more than once might crash the kernel
h.nrn_load_dll(os.path.join(mechanisms_path,'x86_64/.libs/libnrnmech.so'))  # use this line instead in Linux
geo_standard = pd.read_csv(geometry_file,index_col='id')
h.tstop = tstop
h.dt = dt
h.steps_per_ms = 1/h.dt

#### Extra Tools

In [5]:
m=.2
s=0.345
mean, std = lognormal(m=m,s=s) #get underlying normal distribution parameters from lognormal distribution parameters
print(mean, std)

-2.299528896659566 1.1748114608101725


In [6]:
sectionexcspikes=[]
sectioninhspikes = []

##Define cell model and simulation model. Edit method "set_channels" to design biophysical properties.

In [7]:
# from SROL5.build_model import Simulation

In [8]:
class Cell(Stylized_Cell):
    """Define single cell model using parent class Stylized_Cell"""
    def __init__(self,geometry=None,biophys=None,dL=5,vrest=-72.0): #-90? # dL=30 originally
        """
        Initialize cell model
        geometry: pandas dataframe of cell morphology properties
        biophys: vector of biophysical parameters corresponding to "biophys_entries". Use -1 for default value.
        dL: maximum segment length
        vrest: reversal potential for leak channels
        """
        self.biophys = biophys
        super().__init__(geometry,dL,vrest)
        self.record_soma_v() # uncomment this if want to record soma voltage
        self.syn_dist_list = []
        self.gmax_list = []
        self.P_0_list = []
        self.synapse_sec_list = []
    
    def set_channels(self):
        """Define biophysical properties, insert channels"""
        self.define_biophys_entries()
        # common parameters
        for sec in self.all:
            sec.cm = 2.0
            sec.Ra = 100
            sec.insert('pas')
            sec.e_pas = self._vrest
        # fixed parameters
        soma = self.soma
        soma.cm = 1.0           # Originally 1 
        soma.insert('NaTa_t')  # Sodium channel
        soma.insert('SKv3_1')  # Potassium channel
        soma.insert('Ca_HVA')
        soma.insert('Ca_LVAst')
        soma.insert('CaDynamics_E2')
        soma.insert('Ih')
        soma.insert('SK_E2')
        soma.insert('K_Tst')
        soma.insert('K_Pst')
        soma.insert('Nap_Et2')
        soma.ena = 50
        soma.ek = -85
        

        for isec in self.grp_ids[1]:        #prox,mid,dist basal; proxtrunk; oblique
            sec = self.get_sec_by_id(isec) 
            sec.insert('NaTa_t')
            sec.insert('SKv3_1')
            sec.insert('Ca_HVA')
            sec.insert('Ca_LVAst')
            sec.insert('Im')
            sec.insert('CaDynamics_E2')
            sec.insert('Ih')
            sec.insert('SK_E2')
            sec.ena = 50
            sec.ek = -85

        for isec in self.grp_ids[2]:
            sec = self.get_sec_by_id(isec)  # Mid Trunk
            sec.insert('NaTa_t')
            sec.insert('SKv3_1')
            sec.insert('Ca_HVA')
            sec.insert('Ca_LVAst')
            sec.insert('Im')
            sec.insert('CaDynamics_E2')
            sec.insert('Ih')
            sec.insert('SK_E2')
            sec.ena = 50
            sec.ek = -85


        for isec in self.grp_ids[3]:
            sec = self.get_sec_by_id(isec)  # Distal Trunk
            sec.insert('NaTa_t')
            sec.insert('SKv3_1')
            sec.insert('Ca_HVA')
            sec.insert('Ca_LVAst')
            sec.insert('Im')
            sec.insert('CaDynamics_E2')
            sec.insert('Ih')
            sec.insert('SK_E2')
            sec.ena = 50
            sec.ek = -85

        for isec in self.grp_ids[4]:
            sec = self.get_sec_by_id(isec)  # Tuft dendrites
            sec.insert('NaTa_t')
            sec.insert('SKv3_1')
            sec.insert('Ca_HVA')
            sec.insert('Ca_LVAst')
            sec.insert('Im')
            sec.insert('CaDynamics_E2')
            sec.insert('Ih')
            sec.insert('SK_E2')
            sec.ena = 50
            sec.ek = -85


        for isec in self.grp_ids[5]:
            sec = self.get_sec_by_id(isec)  # axon
            sec.cm = 2.0
            sec.insert('NaTa_t')
            sec.insert('SKv3_1')
            sec.insert('Ca_HVA')
            sec.insert('Ca_LVAst')
            sec.insert('Im')
            sec.insert('CaDynamics_E2')
            sec.insert('Ih')
            sec.insert('SK_E2')
            sec.insert('K_Tst')
            sec.insert('K_Pst')
            sec.insert('Nap_Et2')
            sec.ena = 50
            sec.ek = -85
		        
        for isec in self.grp_ids[6]:
            sec = self.get_sec_by_id(isec)  # inactive basal dendrites
            sec.cm = 3.0
            sec.insert('NaTa_t')
            sec.insert('SKv3_1')
            sec.insert('Ca_HVA')
            sec.insert('Ca_LVAst')
            sec.insert('CaDynamics_E2')
            sec.insert('Ih')
            sec.ena = 50
            sec.ek = -85
            # sec.Ra = 100000
            # sec.e_pas = 200

        # variable parameters
        for i,entry in enumerate(self.biophys_entries):
            for sec in self.get_sec_by_id(self.grp_ids[entry[0]]):
                setattr(sec,entry[1],self.biophys[i])
        h.v_init = self._vrest
    

    def define_biophys_entries(self):
        """
        Define list of entries of biophysical parameters.
        Each entry is a pair of group id and parameter reference string.
        Define default values and set parameters in "biophys".
        """
        self.grp_sec_type_ids = [ # select section id's for each group
                                 [0], #soma
                                 [1,2,3,4,5], #basal group: prox,mid,dist basal; proxtrunk; oblique
                                 [6,7,8], #mid trunk,distal trunk, proxtuft
                                 [9], #nexus: midtuft
                                 [10], #tuft: disttuft
                                 [11], #axon
                                 [12] #passive basal dendrites
                                 ]
        self.grp_ids = []  # get indices of sections for each group
        for ids in self.grp_sec_type_ids:
            secs = []
            for i in ids:
                secs.extend(self.sec_id_lookup[i])
            self.grp_ids.append(secs)
        self.biophys_entries = [
            (0,'g_pas'),(1,'g_pas'),(2,'g_pas'),(3,'g_pas'),(4,'g_pas'),(5,'g_pas'),(6,'g_pas'),  # g_pas of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
            (0,'gNaTa_tbar_NaTa_t'),(1,'gNaTa_tbar_NaTa_t'),(2,'gNaTa_tbar_NaTa_t'),(3,'gNaTa_tbar_NaTa_t'),(4,'gNaTa_tbar_NaTa_t'),(5,'gNaTa_tbar_NaTa_t'),(6,'gNaTa_tbar_NaTa_t'),  # gNaTa_t of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
            (0,'gSKv3_1bar_SKv3_1'),(1,'gSKv3_1bar_SKv3_1'),(2,'gSKv3_1bar_SKv3_1'),(3,'gSKv3_1bar_SKv3_1'),(4,'gSKv3_1bar_SKv3_1'),(5,'gSKv3_1bar_SKv3_1'),(6,'gSKv3_1bar_SKv3_1'),  # gSKv3_1 of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
            (0, 'gCa_HVAbar_Ca_HVA'),(1, 'gCa_HVAbar_Ca_HVA'),(2, 'gCa_HVAbar_Ca_HVA'),(3, 'gCa_HVAbar_Ca_HVA'),(4, 'gCa_HVAbar_Ca_HVA'),(5, 'gCa_HVAbar_Ca_HVA'),(6, 'gCa_HVAbar_Ca_HVA'),  # gCA_HVA of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
            (0, 'gCa_LVAstbar_Ca_LVAst'),(1, 'gCa_LVAstbar_Ca_LVAst'),(2, 'gCa_LVAstbar_Ca_LVAst'),(3, 'gCa_LVAstbar_Ca_LVAst'),(4, 'gCa_LVAstbar_Ca_LVAst'),(5, 'gCa_LVAstbar_Ca_LVAst'),(6, 'gCa_LVAstbar_Ca_LVAst'),# gCA_LVAst of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
            (1, 'gImbar_Im'),(2, 'gImbar_Im'),(3, 'gImbar_Im'),(4, 'gImbar_Im'),(5, 'gImbar_Im'), # gIm of basal, midTrunk, distTrunk, tuft, axon
            (0,'decay_CaDynamics_E2'),(1,'decay_CaDynamics_E2'),(2,'decay_CaDynamics_E2'),(3,'decay_CaDynamics_E2'),(4,'decay_CaDynamics_E2'),(5,'decay_CaDynamics_E2'),(6,'decay_CaDynamics_E2'), # decay_CaDynamics of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
            (0,'gamma_CaDynamics_E2'),(1,'gamma_CaDynamics_E2'),(2,'gamma_CaDynamics_E2'),(3,'gamma_CaDynamics_E2'),(4,'gamma_CaDynamics_E2'),(5,'gamma_CaDynamics_E2'),(6,'gamma_CaDynamics_E2'), # gamma_CaDynamics of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
            (0,'gIhbar_Ih'),(1,'gIhbar_Ih'),(2,'gIhbar_Ih'),(3,'gIhbar_Ih'),(4,'gIhbar_Ih'),(5,'gIhbar_Ih'),(6,'gIhbar_Ih'), # gIh of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
            (0,'gSK_E2bar_SK_E2'),(1,'gSK_E2bar_SK_E2'),(2,'gSK_E2bar_SK_E2'),(3,'gSK_E2bar_SK_E2'),(4,'gSK_E2bar_SK_E2'),(5,'gSK_E2bar_SK_E2'), # gSk_E2 of soma, basal, midTrunk, distTrunk, tuft, axon
            (0,'gK_Tstbar_K_Tst'),(5,'gK_Tstbar_K_Tst'), # gK_Tst of soma, axon
            (0,'gK_Pstbar_K_Pst'),(5,'gK_Pstbar_K_Pst'), # gK_Pst of soma, axon
            (0,'gNap_Et2bar_Nap_Et2'),(5,'gNap_Et2bar_Nap_Et2') # gNap_Et2 of soma, axon
        ]

        default_biophys = np.array([0.0000338,0.0000467,0.0000489,0.0000589,0.0000589,0.0000325,0.0000100, # g_pas of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    2.04,0.0213,0.0213,0.0213,0.0213,0.0,0.0, # gNaTa_t of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal # axon was 2.89618
                                    0.693,0.000261,0.000261,0.000261,0.000261,0.0,0.0, # gSKv3_1 of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #axon was 0.473799
                                    0.000992,0.0,0.0000555,0.000555,.0000555,0.0,0.0,  # gCA_HVA of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #basal was 0.000992
                                    0.00343,0.0,0.000187,0.0187,0.000187,0.0,0.0, # gCA_LVAst of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.0000675,0.0000675,0.0000675,0.0000675,0.0, # gIm of soma, basal, midTrunk, distTrunk, tuft, axon
                                    460.0,122,122,122,122,277.300774,122, # decay_CaDynamics of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.000501,0.000509,0.000509,0.000509,0.000509,0.000525,0.000509, # gamma_CaDynamics of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.0002,0.0002,0.0002,0.00507257227,0.01535011884,0.0001,0.0002, # gIh of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.0441,0.0012,0.0012,0.0012,0.0012,0.000047, # gSk_E2 of soma, basal, midTrunk, distTrunk, tuft, axon
                                    0.0812,0.077274, # gK_Tst of soma, axon
                                    0.00223,0.188851, # gK_Pst of soma, axon
                                    0.00172,0.0  # gNap_Et2 of soma, axon
                                    ])

        


        if self.biophys is not None:
            # print('length of default_biophys:',len(default_biophys))
            # print('length of self.biophys:',len(self.biophys))
            for i in range(len(self.biophys)):
                if self.biophys[i]>=0:
                    default_biophys[i]=self.biophys[i]
        self.biophys = default_biophys
    

    def add_synapse(self,stim,sec_index,**kwargs):
        """Add synapse to a section by its index"""
        self.injection.append(Synapse(self,stim,sec_index,**kwargs))
        # self.syn_dist_list.append(self.syn_dist)
        # self.gmax_list.append(self.gmax_list)
        # self.P_0_list.append(self.)
        # self.synapse_sec_list = []
 

    def record_soma_v(self):
        self.v_rec = Recorder(self.soma(.5),'v')
    
    def v(self):
        """Return recorded soma membrane voltage in numpy array"""
        if hasattr(self,'v_rec'):

            return self.v_rec.as_numpy()

class Synapse(Point_current):
    def __init__(self,cell,stim,sec_index,gmax=1,loc=0.5,record=True, SynType='exc',P_0=0.6): 
        super().__init__(cell,sec_index,loc)
        self.stim = stim
        self.gmax = gmax
        self.loc = loc
        self.P_0 = P_0
        self.syntype=SynType
        self.setup(record,loc,P_0,gmax,SynType,stim)

        

    def setup(self,record,loc,P_0,gmax,SynType,stim):
        self.setup_synapse(loc,P_0,gmax,SynType,stim)
        if record:
            self.setup_recorder()
    
    def setup_synapse(self,loc,P_0,gmax,SynType,stim): 
        if SynType == 'inh':
          self.pp_obj = h.int2pyr(self.get_section()(loc))
          #Assigns random generator of release probability.
          r = h.Random()
          r.MCellRan4()
          r.uniform(0,1)
          self.pp_obj.setRandObjRef(r)
          #A list of random generators is kept so that they are not automatically garbaged.
          self.generator = r
          syn = self.pp_obj
          syn.setRandObjRef(r)
          syn.P_0 = P_0
          syn.initW = gmax
          self.nc = h.NetCon(stim,syn,1,0,1)

        else:
          self.pp_obj = h.pyr2pyr(self.get_section()(loc))
          #Assigns random generator of release probability.
          r = h.Random()
          r.MCellRan4()
          r.uniform(0,1)
          self.pp_obj.setRandObjRef(r)
          #A list of random generators is kept so that they are not automatically garbaged.
          self.generator = r
          syn = self.pp_obj
          syn.setRandObjRef(r)
          syn.P_0 = P_0
          syn.initW = gmax
          self.nc = h.NetCon(self.stim,syn,1,0,1)

    def setup_recorder(self):
          size = [round(h.tstop/h.dt)+1] if hasattr(h,'tstop') else []
          try:
              self.rec_vec = h.Vector(*size).record(self.pp_obj._ref_igaba)
          except:
                  self.rec_vec = MultiSynCurrent()
                  vec_inmda = h.Vector(*size).record(self.pp_obj._ref_inmda)
                  vec_iampa = h.Vector(*size).record(self.pp_obj._ref_iampa)
                  self.rec_vec.add_vec(vec_inmda)
                  self.rec_vec.add_vec(vec_iampa)
                  

class MultiSynCurrent(object):
    def __init__(self):
        self.vec_list = []
    
    def add_vec(self,vec):
        self.vec_list.append(vec)
    
    def as_numpy(self):
        return np.sum(np.array([vec.as_numpy() for vec in self.vec_list]), axis=0)


class Simulation(object):
    def __init__(self,geometry,electrodes,loc_param=[0.,0.,0.,0.,1.,0.],
                 geo_param=[-1],biophys=[-1],
                 gmax=0.015,
                 scale=1.0,ncell=1,
                 spike_threshold: Optional[float] = None,
                 synaptic_input=True,
                 RunType='Full Model'): #initially __init__(self,geometry,electrodes,loc_param=[0.,0.,0.,0.,1.,0.],geo_param=[-1],biophys=[-1],gmax=0.015,scale=1.0,ncell=1):
        #exc_dens=2.16,
       #
        # inh_dens=0.22
        """
        Initialize simulation object
        geometry: pandas dataframe of cell morphology properties
        electrodes: array of electrode coordinates, n-by-3
        loc_param: location parameters, ncell-by-6 array, (x,y,z,theta,h,phi)
        geo_param: geometry parameters, ncell-by-k array, if not specified, use default properties in geometry
        biophys: biophysical parameters, ncell-by-k array, if not specified, use default properties
        gmax: maximum conductance of synapse, ncell-vector, if is single value, is constant for all cells
        scale: scaling factors of lfp magnitude, ncell-vector, if is single value, is constant for all cells
        ncell: number of cells in the simulation, required if simulating for multiple cells
        synaptic_input: choose to add synapses to the cell with False, remove with True
        """
        self.ncell = ncell  # number of cells in this simulation
        self.cells = []  # list of cell object
        self.lfp = []  # list of EcpMod object
        self.define_geometry_entries()  # list of entries to geometry dataframe
        self.geometry = geometry.copy()
        self.electrodes = electrodes
        self.set_loc_param(loc_param)
        self.set_geo_param(geo_param)
        self.set_biophys(biophys)
        self.set_gmax(gmax)
        self.set_scale(scale)
        self.create_cells(synaptic_input, RunType)  #exc_dens,inh_dens, create cell objects with properties set up
        self.t_vec = h.Vector( round(h.tstop/h.dt)+1 ).record(h._ref_t)  # record time
        self.spike_threshold = None
        self.record_spikes(spike_threshold)



    def set_netstim(self, syn_number, inhfreq, excfreq, inhFRstd, excFRstd, syntype):
        """Setup spontaneous synaptic input event"""
        stim_list = []
        self.stim = stim_list
        excFRlist=np.random.normal(loc=excfreq, scale=excFRstd, size=syn_number)
        inhFRlist=np.random.normal(loc=inhfreq, scale=inhFRstd, size=syn_number)
        for i in range(syn_number):
        #   if syntype[i] == 'inh':
            if syntype == 'inh':
                frequency = inhfreq
                spikeinterval = (1/frequency)*1000 #convert to period and ms
                stim = h.NetStim()
                stim.number = 1000 #2  # average number of spikes #check that this does not affect spike freq. hopefully only affects duration of stim. 
                #max frequency input for L5 is 16 hz meaning stim.number should be at least tstop/30, but minimize number for better runtime
                stim.start = 50 #(np.random.rand()*50)+50 #startspikes #50  #  most likely start time of first spike
                stim.noise = 1  # fractional randomness
                stim.interval =  spikeinterval #500  # mean time between spikes (ms)
                #duration of stimulation = stim.interval X stim.number
                stim_list.append(stim)
            elif syntype == 'exc':
                frequency = excfreq
                spikeinterval = (1/frequency)*1000 #convert to period and ms
                stim = h.NetStim()
                stim.number = 1000 #2  # average number of spikes #check that this does not affect spike freq. hopefully only affects duration of stim. 
                #max frequency input for L5 is 16 hz meaning stim.number should be at least tstop/30, but minimize number for better runtime
                stim.start = 50 #(np.random.rand()*50)+50 #startspikes #50  #  most likely start time of first spike
                stim.noise = 1  # fractional randomness
                stim.interval =  spikeinterval #500  # mean time between spikes (ms)
                #duration of stimulation = stim.interval X stim.number
                stim_list.append(stim)
    
    def set_exc_grp_netstim(self, syn_number, excfreq, excFRstd, loc, grpx, clusterx, writer):
        """
        Setup clustered synaptic exc input events for a section.
        syn_number: the number of synapses
        excfreq: mean of normal distribution representing firing rates from excitatory afferent cells
        excFRstd: standard deviation of normal distribution representing firing rates from excitatory afferent cells
        loc: array of size syn_number denoting 0<x<1 normalized location of synapse along section. ex: [0.987, 0.124, 0.634, 0.223, 0.788, ...]
        grpx: array denoting functional group normalized location bounds along section. ex: [0,0.5,1]
        clusterx: array denoting cluster normalized location bounds along section. ex: [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
        """
        stim_list = np.empty(syn_number, dtype=object)
        self.stim = stim_list
        ##may need to double check that these values are truly only exc if inh equivalent is unused
        mean_fr=excfreq
        std_fr=excFRstd
        numclusters=len(clusterx)-1
        numgrps=len(grpx)-1
        excfreqs=[]
        ExcSpikeMatrix=[]
        #distribution used for generating excitatory firing rates.    
        levy_dist = partial(st.levy_stable.rvs, alpha=1.37, beta=-1.00, loc=0.92, scale=0.44, size=1)
        # a, b = (0 - mean_fr) / std_fr, (100 - mean_fr) / std_fr #(0 - mean_fr) / std_fr, (100 - mean_fr) / std_fr
        # levy_dist = partial(st.levy_stable.rvs, alpha=a, beta=b, loc=mean_fr, scale=std_fr) #experiment
        #normal distribution of cell firing rates
        exc_dist=levy_dist(size=numclusters)
        # exc_dist=np.clip(exc_dist, 0,None)
        # print(min(exc_dist))
        #buffer = self.params["time"]["start"]
        for k in range(numgrps): #iterate along functional groups
          z = make_noise(num_samples=(int(tstop))-1,num_traces=1) #generate the noise trace common to the functional group. tstop was originally seconds*1000, num samples should be length of trace
          rate_prof=np.tile(z[0,:],(numclusters,1))
          # print(rate_prof)
          for m in range(numclusters): #iterate through number of clusters
            if (clusterx[m]>=grpx[k]) and (clusterx[m+1]<=grpx[k+1]): #get new single stim for cluster #may cause issue where cluster bounds surround a group edge instead of being within a group.
              rate_temp=[];simSpks_temp=[]
              r=rate_prof[m,:] # get cluster's noise rate profile
              r[r<0] = 0 #Can't have negative firing rates.
              rate_temp = r*(exc_dist[m]) #pull excfr from noise-modulated fr distribution
              rate_temp[rate_temp<0] = 0
              # print(rate_temp/1000))
              #generate spikes given fr
              numbPoints = st.poisson(rate_temp/1000).rvs()#Poisson number of points
              simSpks=np.where(numbPoints>0)[0]
              #create stim
              excspikes=simSpks
              if simSpks is None:
                excspikes=[]
              # NRN will fail if VecStim contains negative spike-time, throw an exception and log info for user
              #print('spike train {} contains negative number, unable to run virtual cell in NEURON'.format(excspikes))
              #get same stim for exc synapses in cluster
              excspikes = np.sort(excspikes)
              # ExcSpikeMatrix.append(excspikes)    # spike train per cluster Gathering spike trains for printing a raster
              inputfreq=(len(excspikes)/(tstop/1000)) # print the number of spikes array. dividing by tstop should give the spike frequency
              excfreqs.append(int(inputfreq))
              self._exc_train_vec = h.Vector(excspikes)
              excvecstim = h.VecStim()
              excvecstim.play(self._exc_train_vec)
              self._hobj = excvecstim
              # print(excvecstim)
              for i in range(syn_number):
                #make sure synapse is within current functional group
                #if (loc[i]>=grpx[k]) and (loc[i]<=grpx[k+1]): #not necessary if you know current cluster is within current group already
                  #assign stim to all synapses within current cluster
                  if (loc[i]>=clusterx[m]) and (loc[i]<=clusterx[m+1]): #where location is between current group bound
                      writer.append_repeat("node_ids", i, len(simSpks))
                      writer.append_ds(simSpks, "timestamps")
                      stim_list[i]=excvecstim
                      ExcSpikeMatrix.append(excspikes)    #spike train per synapse Gathering spike trains for printing a raster
        sectionexcspikes.append(ExcSpikeMatrix)
        # print(excfreqs)


    def set_inh_grp_netstim(self, syn_number, inhfreq, inhFRstd, loc, rhythmic_inh=True):
        """
        Setup clustered synaptic input events
        syn_number: the number of synapses
        inhfreq/excfreq: mean inh/exc FR
        inhFRstd/excFRstd: standard deviation of inh/exc FR
        syntype: array of size syn_number containing 'inh' or 'exc'
        loc: array of size syn_number containing 0<x<1 normalized location of synapse along section
        """
        stim_list = []
        self.stim = stim_list
        ##may need to double check that these values are truly only exc if inh equivalent is unused
        mean_fr=inhfreq
        std_fr=inhFRstd
        inhfreqs=[]
        InhSpikeMatrix=[]
        # #distribution used for generating excitatory firing rates.    
        # exc_dist = partial(st.levy_stable.rvs, alpha=1.37, beta=-1.00, loc=0.92, scale=0.44, size=1)
        #normal distribution of cell firing rates
        a, b = (0 - mean_fr) / std_fr, (100 - mean_fr) / std_fr #(0 - mean_fr) / std_fr, (100 - mean_fr) / std_fr
        d = partial(st.truncnorm.rvs, a=a, b=b, loc=mean_fr, scale=std_fr) #Normal Distribution of cells
        #buffer = self.params["time"]["start"]
        # for k in range(numgrps): #iterate along functional groups
          #generate the noise trace common to the functional group.
        #   z = make_noise(num_samples=(int(tstop))-1,num_traces=1) # tstop was originally seconds*1000, num samples should be length of trace
        if rhythmic_inh: # make an array of modulated sin waves
          frs = d(size=syn_number)
          # frs=np.clip(frs, 0,3*mean_fr)
          # t = np.arange(0,tstop,0.001) #original
          t = np.arange(0,tstop-1,1) #test
          z = np.zeros((syn_number,t.shape[0]))
          P = 0
          for i in np.arange(0,syn_number):
            offset = frs[i]
            rhythmic_mod=0.5
            rhythmic_f=16
            A = offset/((1/rhythmic_mod)-1)
            z[i,:] = A*np.sin((2 * np.pi * rhythmic_f * t)+P) + offset
            dist=np.ones((syn_number,1))
        else: # create a noise trace by shifting exc spikes #
            f = h5py.File("exc_stim_spikes.h5", "r")
            ts = f['spikes']["exc_stim"]['timestamps'] #ts can be SimSpks from saving excitatory if the
            nid = f['spikes']["exc_stim"]['node_ids']
            z = shift_exc_noise(ts, nid, tstop, time_shift=2) #z = shift_exc_noise(ts, nid, self.params["time"]["stop"], time_shift=self.params["inh_shift"])
            z = np.tile(z,(syn_number,1))
            dist=d(size=syn_number)
        for m in range(syn_number):
            rate_temp=[];simSpks_temp=[]
            rate_prof=np.tile(z[0,:],(syn_number,1))
            r=rate_prof[m,:]
            r[r<0] = 0#Can't have negative firing rates.
            #pull inhfr from modulated fr distribution
            rate_temp = r*dist[m]
            #generate spikes given fr
            numbPoints = st.poisson(rate_temp/1000).rvs()#Poisson number of points
            simSpks=np.where(numbPoints>0)[0]
            #create stim
            inhspikes=simSpks
            if simSpks is None:
              inhspikes=[]
            inputfreq=(len(inhspikes)/(tstop/1000)) # print the number of spikes array. dividing by tstop should give the spike frequency
            inhfreqs.append(int(inputfreq))
            # NRN will fail if VecStim contains negative spike-time, throw an exception and log info for user
            #print('spike train {} contains negative number, unable to run virtual cell in NEURON'.format(inhspikes))
            #get same stim for inh synapse in cluster
            #get same stim for inh synapses in cluster
            inhspikes = np.sort(inhspikes)
            # InhSpikeMatrix.append(inhspikes)
            self._inh_train_vec = h.Vector(inhspikes)
            inhvecstim = h.VecStim()
            inhvecstim.play(self._inh_train_vec)
            # writer.append_repeat("node_ids", i, len(simSpks))
            # writer.append_ds(simSpks, "timestamps")
            stim_list.append(inhvecstim)
            InhSpikeMatrix.append(inhspikes)
        sectioninhspikes.append(InhSpikeMatrix)
        # print(inhfreqs)


    
    def pack_parameters(self,param,ndim,param_name):
        """Pack parameters for the simulation"""
        if ndim==0:
            if not hasattr(param,'__len__'):
                param = [param]
            param = np.array(param).ravel()
            if param.size!=self.ncell:
                if param.size==1:
                    param = np.broadcast_to(param,self.ncell)
                else:
                    raise ValueError(param_name+" size does not match ncell")   
        if ndim==1:
            param = np.array(param)
            if param.ndim==1:
                param = np.expand_dims(param,0)
            if param.shape[0]!=self.ncell:
                if param.shape[0]==1:
                    param = np.broadcast_to(param,(self.ncell,param.shape[1]))
                else:
                    raise ValueError(param_name+" number of rows does not match ncell")
        return param
    
    def set_loc_param(self,loc_param):
        """Setup location parameters. loc_param ncell-by-6 array"""
        loc_param = self.pack_parameters(loc_param,1,"loc_param")
        self.loc_param = [(loc_param[i,:3],loc_param[i,3:]) for i in range(self.ncell)]
    
    def set_geo_param(self,geo_param):
        """Setup geometry parameters. geo_param ncell-by-k array, k entries of properties"""
        self.geo_param = self.pack_parameters(geo_param,1,"geo_param")
    
    def set_biophys(self,biophys):
        """Setup geometry parameters. geo_param ncell-by-k array, k entries of properties"""
        self.biophys = self.pack_parameters(biophys,1,"biophys")
    
    # def set_gmax(self,gmaxEXC,gmaxINH):
    #     """Setup maximum conductance of synapse"""
    #     self.gmaxEXC = self.pack_parameters(gmaxEXC,0,"gmaxEXC")
    #     self.gmaxINH = self.pack_parameters(gmaxINH,0,"gmaxINH")
    def set_gmax(self,gmax):
        """Setup maximum conductance of synapse"""
        self.gmax = self.pack_parameters(gmax,0,"gmax")
    
    def set_scale(self,scale):
        """setup scaling factors of lfp magnitude"""
        self.scale = self.pack_parameters(scale,0,"scale")
    
    def define_geometry_entries(self):
        """Define list of entries to geometry dataframe. Each entry is a pair of section id and property."""
        self.geo_entries = [
            #soma
            (0,'R'),  # radius
        #     #proximal trunk
        #     (4,'excitation'), # excitation ('TRUE'/'FALSE')
        #     (4,'inhibition'), # inhibition ('TRUE'/'FALSE')
        #     (4,'L'), # length
        #     (4,'R'), # radius
        #     (4,'excFRmean'), # exc FR mean
        #     (4,'excFRstd'), # exc FR std
        #     (4,'inhFRmean'), # inh FR mean
        #     (4,'inhFRstd'), # inh FR std
        #     (4,'excP_0mean'), # exc P_0 mean (0 to 1)
        #     (4,'excP_0std'), # exc P_0 std
        #     (4,'inhP_0mean'), # inh P_0 mean (0 to 1)
        #     (4,'inhP_0std'), # inh P_0 std
        #     #middle trunk
        #     (6,'excitation'), # excitation ('TRUE'/'FALSE')
        #     (6,'inhibition'), # inhibition ('TRUE'/'FALSE')
        #     (6,'L'), # length
        #     (6,'R'), # radius
        #     (6,'excFRmean'), # exc FR mean
        #     (6,'excFRstd'), # exc FR std
        #     (6,'inhFRmean'), # inh FR mean
        #     (6,'inhFRstd'), # inh FR std
        #     (6,'excP_0mean'), # exc P_0 mean (0 to 1)
        #     (6,'excP_0std'), # exc P_0 std
        #     (6,'inhP_0mean'), # inh P_0 mean (0 to 1)
        #     (6,'inhP_0std'), # inh P_0 std
        #     #distal trunk
        #     (7,'excitation'), # excitation ('TRUE'/'FALSE')
        #     (7,'inhibition'), # inhibition ('TRUE'/'FALSE')
        #     (7,'L'), # length
        #     (7,'R'), # radius
        #     (7,'excFRmean'), # exc FR mean
        #     (7,'excFRstd'), # exc FR std
        #     (7,'inhFRmean'), # inh FR mean
        #     (7,'inhFRstd'), # inh FR std
        #     (7,'excP_0mean'), # exc P_0 mean (0 to 1)
        #     (7,'excP_0std'), # exc P_0 std
        #     (7,'inhP_0mean'), # inh P_0 mean (0 to 1)
        #     (7,'inhP_0std'), # inh P_0 std
        #     #basal dendrites
        #     ([1,2,3],'excitation'), # excitation ('TRUE'/'FALSE')
        #     ([1,2,3],'inhibition'), # inhibition ('TRUE'/'FALSE')
        #     ([1,2,3],'nbranch'), # number of dendrites
        #     (1,'L'), # proximal section length
        #     (2,'L'), # middle section length
        #     (3,'L'), # distal section length
        #     (1,'R'), # proximal radius
        #     (2,'R'), # middle radius
        #     (3,'R'), # distal radius
        #     ([1,2,3],'ang'), # angle
        #     ([1,2,3],'excFRmean'), # exc FR mean
        #     ([1,2,3],'excFRstd'), # exc FR std
        #     ([1,2,3],'inhFRmean'), # inh FR mean
        #     ([1,2,3],'inhFRstd'), # inh FR std
        #     ([1,2,3],'excP_0mean'), # exc P_0 mean (0 to 1)
        #     ([1,2,3],'excP_0std'), # exc P_0 std
        #     ([1,2,3],'inhP_0mean'), # inh P_0 mean (0 to 1)
        #     ([1,2,3],'inhP_0std'), # inh P_0 std
        #     #oblique dendrites
        #     (5,'excitation'), # excitation ('TRUE'/'FALSE')
        #     (5,'inhibition'), # inhibition ('TRUE'/'FALSE')
        #     (5,'nbranch'), # number of dendrites
        #     (5,'L'), # length
        #     (5,'R'), # radius
        #     (5,'ang'), # angle
        #     (5,'excFRmean'), # exc FR mean
        #     (5,'excFRstd'), # exc FR std
        #     (5,'inhFRmean'), # inh FR mean
        #     (5,'inhFRstd'), # inh FR std
        #     (5,'excP_0mean'), # exc P_0 mean (0 to 1)
        #     (5,'excP_0std'), # exc P_0 std
        #     (5,'inhP_0mean'), # inh P_0 mean (0 to 1)
        #     (5,'inhP_0std'), # inh P_0 std
        #     # tuft dendrites
        #     ([8,9,10],'excitation'), # excitation ('TRUE'/'FALSE')
        #     ([8,9,10],'inhibition'), # inhibition ('TRUE'/'FALSE')
        #     ([8,9,10],'nbranch'), # number of dendrites
        #     (8,'L'), # proximal section length
        #     (9,'L'), # middle section length
        #     (10,'L'), # distal section length
        #     (8,'R'), # proximal radius
        #     (9,'R'), # middle radius
        #     (10,'R'), # distal radius
        #     ([8,9,10],'ang'), # angle
        #     ([8,9,10],'excFRmean'), # exc FR mean
        #     ([8,9,10],'excFRstd'), # exc FR std
        #     ([8,9,10],'inhFRmean'), # inh FR mean
        #     ([8,9,10],'inhFRstd'), # inh FR std
        #     ([8,9,10],'excP_0mean'), # exc P_0 mean (0 to 1)
        #     ([8,9,10],'excP_0std'), # exc P_0 std
        #     ([8,9,10],'inhP_0mean'), # inh P_0 mean (0 to 1)
        #     ([8,9,10],'inhP_0std'), # inh P_0 std
        #     #axon
        #     (11,'L'), # length
        #     (11,'R'), # radius
        #     #passive basal
        #     (12,'nbranch'), # number of dendrites
        #     (12,'L'), # length
        #     (12,'R') # radius
        ]
    
    def set_geometry(self,geometry,geo_param):
        """Set property values from geo_param through each entry to geometry. Return dataframe"""
        geom = geometry.copy()
        for i,x in enumerate(geo_param):
            if x>=0:
                geom.loc[self.geo_entries[i]] = x
        return geom

    def record_spikes(self, threshold: Optional[float]) -> None:
        """Setup spike recorder for all cells"""
        for cell in self.cells:
            cell.set_spike_recorder(threshold)
        self.spike_threshold = threshold

    def create_cells(self, synaptic_input, RunType): #exc_dens,inh_dens
        """Create cell objects with properties set up"""
        self.cells.clear()  # remove cell objects from previous run
        self.lfp.clear()
        # Create cell with morphology and biophysical parameters
        for i in range(self.ncell):
            geometry = self.set_geometry(self.geometry,self.geo_param[i,:])
            self.cells.append( Cell(geometry=geometry,biophys=self.biophys[i,:]) )
        # add injection current or synaptic current and set up lfp recording
        min_dist = 10.0 # minimum distance allowed between segment and electrode. Set to None if not using.
        for i,cell in enumerate(self.cells):
            self.lfp.append( EcpMod(cell,self.electrodes,move_cell=self.loc_param[i],scale=self.scale[i],min_distance=min_dist) )
            #Current injection in soma
            # cell.add_injection(sec_index=19,record=True,delay=150,dur=150,amp=0.1) # Tune for proper action potential
            if RunType == 'Active Properties':
              print('Active Property Simulation')
              cell.add_injection(sec_index=0,record=True,delay=250,dur=750,amp=0.1) # Tune for proper action potential 
              cell.add_injection(sec_index=0,record=True,delay=1250,dur=750,amp=0.2) 
              cell.add_injection(sec_index=0,record=True,delay=2250,dur=750,amp=0.3) 
              cell.add_injection(sec_index=0,record=True,delay=3250,dur=750,amp=0.4) 
            if RunType == 'Passive Properties':
              print('Passive Property Simulation')
              cell.add_injection(sec_index=0,record=True,delay=150,dur=850,amp=-0.1) # Tune for proper action potential 
            if synaptic_input == True:
              writer = SonataWriter('exc_stim_spikes.h5', ["spikes", "exc_stim"], ["timestamps", "node_ids"], [float, int])
              for id,sec in self.geometry.iterrows(): #add exc synapses
                  sec_index_list=cell.sec_id_lookup[id]
                  if sec['excitation']:
                    L = sec['L']
                    excitation=sec['excitation']
                    #Clustering
                    funcgrouplength=100 #desired functional group length (um)      #warning: rounding may alter
                    clusterlength=3 #desired cluster length (um)                    #warning: rounding may alter
                    #calculate number of functional groups and clusters on this section
                    funcgrps=round(L/funcgrouplength) #calculate the number of functional groups for this section
                    clusters=int(np.floor(L/clusterlength)) #calculate the number of clusters for this section
                    if funcgrps==0:
                      funcgrps=1
                    if clusters==0:
                      clusters=1
                    print(sec['name'],'|L:',L, '|cluster length:', L/clusters, '|# clusters:',clusters,'|funcgrp length:', L/funcgrps, '|# funcgrp:',funcgrps, '|syns/cluster:', round(2.16*L/clusters),'|#segments:',math.ceil(L/5),'|segment length:',L/math.ceil(L/5))
                    #print('The number of clusters is ',clusters)
                    #calculate the bounds of functional groups on this section
                    grpx=[]
                    for num in range(funcgrps):
                      grpx.append(num/funcgrps) # construct an array of functional group bounds where x denotes a proportion (0.0 to 1.0) of the section length
                    grpx.append(1)
                    #print('grpx for ',sec['name'],funcgrps,' is ',grpx)
                    #calculate the bounds of clusters on this section
                    clusterx=[]
                    for num in range(clusters):
                      clusterx.append(num/clusters) # construct an array of cluster bounds where x denotes a proportion (0.0 to 1.0) of the section length
                    clusterx.append(1)
                    #print('clusterx for ',sec['name'],clusters,' is ',clusterx)
                    #print(funcgrps)

                    #pull section synaptic parameters
                    excP_0mean=sec['excP_0mean']
                    excP_0std=sec['excP_0std']
                    excFRmean=sec['excFRmean']
                    excFRstd=sec['excFRstd']
                    exc_dens=sec['exc_dens']

                    # #print('Distributing exc synapses onto',sec['name'],'sections')
                    # syn_exc_number = np.round((exc_dens)*L) # number of synapses on dendrites = density (syn/um) times length of dendrites (um)
                    syn_exc_number = np.round((exc_dens)*L*excP_0mean) # number of synapses on dendrites = density (syn/um) times length of dendrites (um)
                    syn_number = int(syn_exc_number) #total number of synapses on the section
                    for isec in range(len(sec_index_list)):
                        #print(sec['name'],'has ',funcgrps,' groups and ',syn_number,' synapses')
                        syn_dist = np.random.rand(syn_number) #generate random values for 'x' normalized location along section
                        gmax_dist = np.random.lognormal(mean, std, syn_number) #generate random values for gmax
                        gmax_dist=np.clip(gmax_dist,0,5)
                        self.set_exc_grp_netstim(syn_number,excfreq=excFRmean,excFRstd=excFRstd, loc=syn_dist, grpx=grpx, clusterx=clusterx, writer=writer) #generate exc presynaptic spike trains
                        # P_0exclist=np.random.normal(loc=excP_0mean, scale=excP_0std, size=syn_number)
                        P_0exclist=np.ones(syn_number)
# can adjust weights ############################################
                        if id == 2: #adjust midbasal weight
                          tune=1 #consider 1.5/1.1
                        elif id == 3: #adjust distbasal weight
                          tune=1 #consider 5/3
                        else: # adjust rest excitation
                          tune=1
#################################################################
                        for j in range(syn_number): #add synapses to this section
                            cell.add_synapse(stim=self.stim[j],sec_index=sec_index_list[isec],gmax=gmax_dist[j],loc=syn_dist[j],SynType='exc',P_0=P_0exclist[j])
                            # cell.add_synapse(stim=self.stim[j],sec_index=sec_index_list[isec],gmax=gmax_dist[j]*(TRUNCNORM_MAX_BRENT_ITERS),loc=syn_dist[j],SynType='exc',P_0=P_0exclist[j])
                  else: 
                      print(sec['name'],'has no exc synapses')
                      for isec in range(len(sec_index_list)):
                        sectionexcspikes.append([])

              for id,sec in self.geometry.iterrows(): #add inh synapses
                  sec_index_list=cell.sec_id_lookup[id]
                  inhibition=sec['inhibition']
                  if sec['inhibition']:
                    L = sec['L']
                    #pull section synaptic parameters for section
                    inhP_0mean=sec['inhP_0mean']
                    inhP_0std=sec['inhP_0std']
                    inhFRmean=sec['inhFRmean']
                    inhFRstd=sec['inhFRstd']
                    inh_dens=sec['inh_dens']
                    rhythmic_inh=sec['rhythmic_inh']
                    random.seed(427+(id*id))
                    if id==0: # soma ### add specific inhibition to soma where length=0
                      syn_number = 406
                    else:
                        # syn_inh_number = np.round((inh_dens)*L) # number of synapses on dendrites = density (syn/um) times length of dendrites (um)
                        # syn_number = int(syn_inh_number) #total number of synapses on the section
                        syn_inh_number = np.round((inh_dens)*L*inhP_0mean) # number of synapses on dendrites = density (syn/um) times length of dendrites (um)
                        syn_number = int(syn_inh_number) #total number of synapses on the section
                    print('Distributing',syn_number,'inh synapses onto',sec['name'],'sections')
                    for isec in range(len(sec_index_list)):
                        #print(sec['name'],'has ',funcgrps,' groups and ',syn_number,' synapses')
                        syn_dist = np.random.rand(syn_number) #generate random values for 'x' normalized location along section
                        # self.set_inh_grp_netstim(syn_number,inhfreq=inhFRmean,inhFRstd=inhFRstd, loc=syn_dist, rhythmic_inh=rhythmic_inh) #generate inh presynaptic spike trains
                        self.set_inh_grp_netstim(syn_number,inhfreq=inhFRmean,inhFRstd=inhFRstd, loc=syn_dist, rhythmic_inh=False) #generate inh presynaptic spike trains
                        # P_0inhlist=np.random.normal(loc=inhP_0mean, scale=inhP_0std, size=syn_number)
                        P_0inhlist=np.ones(syn_number)
# can adjust weights ############################################
                        if id==0: #soma
                          tune=1
                        elif id==3: #distbasal
                          tune=1
                        elif id==10: #disttuft
                          tune=1 #5/3
                        elif id==9: #mid tuft
                          tune=1 #4.5/4
                        elif id==8: #prox tuft
                          tune=1
                        else: #elsewhere
                          tune=1
#################################################################
                        for j in range(syn_number): #add synapses
                          cell.add_synapse(stim=self.stim[j],sec_index=sec_index_list[isec],gmax=1*tune,loc=syn_dist[j],SynType='inh',P_0=P_0inhlist[j]) #elsewhere
                  else: 
                      print(sec['name'],'has no inh synapses')
                      for isec in range(len(sec_index_list)):
                        sectioninhspikes.append([])
                      
            else:
                print('Creating Passive Cell')

        # # Old implementation #Add synapses without rhythmic inhibition/input grouping
        # for i,cell in enumerate(self.cells):
        #   self.lfp.append( EcpMod(cell,self.electrodes,move_cell=self.loc_param[i],scale=self.scale[i],min_distance=min_dist) )
        #   # Current injection in soma
        #   #cell.add_injection(sec_index=19,record=True,delay=150,dur=150,amp=0.1) # Tune for proper action potential 
        #   for id,sec in self.geometry.iterrows(): #add exc synapses
        #       sec_index_list=cell.sec_id_lookup[id]
        #       L = sec['L']
        #       #pull section synaptic parameters
        #       excP_0mean=sec['excP_0mean']
        #       excP_0std=sec['excP_0std']
        #       inhP_0mean=sec['inhP_0mean']
        #       inhP_0std=sec['inhP_0std']
        #       excFRmean=sec['excFRmean']
        #       excFRstd=sec['excFRstd']
        #       inhFRmean=sec['inhFRmean']
        #       inhFRstd=sec['inhFRstd']
        #       inhibition=sec['inhibition']
        #       excitation=sec['excitation']
        #       inh_dens=sec['inh_dens']
        #       exc_dens=sec['exc_dens']
        #       rhythmic_inh=sec['rhythmic_inh']

        #       if id==0: # soma
        #         #print('soma skipped') #skip soma
        #         ### add specific inhibition to soma
        #         syn_number = 406
        #         #print(self.stim)
        #         syn_dist = np.random.rand(syn_number) #generate distribution for location
        #         syn_type_list = ['inh' for y in range(syn_number)] #assign randomly, adhere to proportion
        #         #print(syn_type_list)
        #         self.set_netstim(syn_number,inhfreq=inhFRmean,excfreq=excFRmean,excFRstd=excFRstd,inhFRstd=inhFRstd,syntype=syn_type_list)
        #         # print(self.stim)
        #         P_0inhlist=np.random.normal(loc=inhP_0mean, scale=inhP_0std, size=syn_number)
        #         P_0exclist=np.random.normal(loc=excP_0mean, scale=excP_0std, size=syn_number)
        #         for j in range(syn_number):
        #           cell.add_synapse(stim=self.stim[j],sec_index=sec_index_list[0],gmax=1,loc=syn_dist[j],SynType=syn_type_list[j],P_0=P_0inhlist[j])

        #       else: # add synapses to each section
        #           if sec['excitation']:
        #               syn_exc_number = np.round((exc_dens)*L) # number of synapses on dendrites = density (syn/um) times length of dendrites (um)
        #           else: 
        #               syn_exc_number=0
        #           if sec['inhibition']:
        #               syn_inh_number = np.round((inh_dens)*L)
        #           else:
        #               syn_inh_number=0
        #           if (not sec['inhibition']) and (not sec['excitation']):
        #               print(sec['name'],'has no synapses')
        #           else:
        #               syn_number = int(syn_inh_number + syn_exc_number) #total number of synapses on the section
        #               prop_exc = syn_exc_number/syn_number #used for assigning type
        #               for isec in range(len(sec_index_list)):
        #                   #print(sec['name'],'has ',funcgrps,' groups and ',syn_number,' synapses')
        #                   syn_dist = np.random.rand(syn_number) #generate random values for 'x' normalized location along section
        #                   rand_syntype = np.random.rand(syn_number)  #generate random values for determining type
        #                   gmax_dist = np.random.lognormal(mean, std, syn_number) #generate random values for gmax
        #                   syn_type_list = ['exc' if y< prop_exc else #generate list of synapse type by assigning type using random values 
        #                                   'inh' for y in rand_syntype]
        #                   self.set_netstim(syn_number,inhfreq=inhFRmean,excfreq=excFRmean,excFRstd=excFRstd,inhFRstd=inhFRstd,syntype=syn_type_list) #generate list of presynaptic spike trains
        #                   P_0exclist=np.random.normal(loc=excP_0mean, scale=excP_0std, size=syn_number)
        #                   P_0inhlist=np.random.normal(loc=inhP_0mean, scale=inhP_0std, size=syn_number)
        #                   for j in range(syn_number): #add synapses using list indices
        #                     if syn_type_list[j] == 'exc':
        #                       cell.add_synapse(stim=self.stim[j],sec_index=sec_index_list[isec],gmax=gmax_dist[j],loc=syn_dist[j],SynType=syn_type_list[j],P_0=P_0exclist[j])
        #                     elif syn_type_list[j] == 'inh':
        #                       cell.add_synapse(stim=self.stim[j],sec_index=sec_index_list[isec],gmax=1,loc=syn_dist[j],SynType=syn_type_list[j],P_0=P_0inhlist[j])
        #                     else:
        #                       print('Error: syn_type not exc/inh')
          

        

    
    def run(self):
        """Run simulation"""
        h.run()
    
    def t(self):
        """Return simulation time vector"""
        return self.t_vec.as_numpy()
    
    def get_lfp(self,index=0):
        """Return LFP array of the cell by index (indices), (cells-by-)channels-by-time"""
        if not hasattr(index,'__len__'):
            lfp = self.lfp[index].calc_ecp()
        else:
            index = np.asarray(index).ravel()
            lfp = np.stack([self.lfp[i].calc_ecp() for i in index],axis=0)
        return lfp

    def get_spike_time(self, index: Union[np.ndarray, List[int], int, str] = 0) -> np.ndarray:
        """
        Return soma spike time of the cell by index (indices), ndarray (list of ndarray)
        Parameters
        index: index of the cell to retrieve the spikes from
        """
        if self.spike_threshold is None:
            raise ValueError("Spike recorder was not set up.")
        if type(index) is str and index == 'all':
            index = range(self.ncell)
        if not hasattr(index, '__len__'):
            spk = self.cells[index].spikes.as_numpy().copy()
        else:
            index = np.asarray(index).ravel()
            spk = np.array([self.cells[i].spikes.as_numpy().copy() for i in index], dtype=object)
        return spk

    def get_spike_number(self, index: Union[np.ndarray, List[int], int, str] = 0) -> Union[int, np.ndarray]:
        """
        Return soma spike number of the cell by index (indices), int (ndarray)
        Parameters
        index: index of the cell to retrieve the spikes from
        """
        if self.spike_threshold is None:
            raise ValueError("Spike recorder was not set up.")
        if index == 'all':
            index = range(self.ncell)
        if not hasattr(index, '__len__'):
            spk = self.get_spike_time(index)
            nspk = spk.size
        else:
            index = np.asarray(index).ravel()
            spk = self.get_spike_time(index)
            nspk = np.array([s.size for s in spk])
        return nspk, 

In [9]:
# add axial current to parent section by computing middle of section's first segment, and doing .ri(), and recording voltage at .ri() loc and section.parentseg()

In [10]:
# class Record_Axial_Current(object):
#     """A module for recording axial currents from section object to segments attached to section object"""
#     def __init__(self, section: nrn.Section, dend_type: Optional[str] = None, record_t: bool = False, single_seg: bool = False) -> None:
#         """
#         section: section object
#         dend_type: list of section names of the dendrite types that need to be recorded
#         record_t: whether or not to record time points
#         single_seg: whether or not to record only one segment for each dendrite type
#         """
#         self.section = section
#         if dend_type is None:
#             # import pdb; pdb.set_trace()
#             sec_names = [sec.name().split('.')[-1].split('[')[0] for sec in section.children()]
#             if section.parentseg() is not None: # check if section has a parent section
#               sec_names.append(section.parentseg().sec.name().split('.')[-1].split('[')[0]) #get name of parent section
#             self.dend_type = list(set(sec_names))
#         else:
#             self.dend_type = dend_type
#         self.dend = {};
#         # import pdb; pdb.set_trace()
#         for d in self.dend_type:
#             self.dend[d] = Adjacent_Section(self.section,d)
#         self.single_seg = single_seg
#         self.setup_recorder(record_t)
        
    
#     def setup_recorder(self, record_t: bool = False):
#         if record_t:
#             self.t_vec = h.Vector(round(h.tstop / h.dt) + 1).record(h._ref_t)
#         else:
#             self.t_vec = None
#         for dend in self.dend.values():
#             dend.setup_recorder(self.single_seg)
    
#     def t(self):
#         if self.t_vec is None:
#             t = None
#         else:
#             t = self.t_vec.as_numpy().copy()
#         return t
    
#     def get_current(self, dend_type: Optional[str] = None) -> np.ndarray:
#         if dend_type is None:
#             axial_current = {}
#             for name,dend in self.dend.items():
#                 axial_current[name] = dend.get_current()
#         else:
#             axial_current = self.dend[dend_type].get_current()
#         return axial_current

# class Adjacent_Section(object):
#     """A module for recording and calculating axial current from the soma to its adjacent sections of a dendrite type"""
#     def __init__(self, section: nrn.Section, name: Optional[str] = 'dend') -> None:
#         """
#         section: section section object
#         name: section names of the dendrite type
#         """
#         self.name = name
#         self.section=section
#         #calculate section children seg to section seg
#         self.init_sec = [s for s in section.children() if name in s.name()] #get array of section children and check if they are in the sec_names list

#         if (section.parentseg() is not None):
#           if (name == section.parentseg().sec.name().split('.')[-1].split('[')[0]): # In this case we have passed no children sections and the name of the parent section is passed
#             self.init_sec = [section.parentseg().sec] #set parent section as initial section
#             self.init_seg= section.parentseg() # set inital segment as parent segment
#         # import pdb; pdb.set_trace()
#         self.nseg = [s.nseg for s in self.init_sec]
#         self.init_seg = [s(0.5/n) for s,n in zip(self.init_sec,self.nseg)] #calculate middle of the first segment in the section for every child section
#         # #calculate section to parent section
#         # self.parent_seg=section.parentseg()
#         # self.first_seg=section(0.5/section.nseg)
    
#     def setup_recorder(self, single_seg: bool = False):
#         self.pre_seg = [s.parentseg() for s in self.init_sec]
#         if (self.section.parentseg() is not None):
#           if self.init_sec == [self.section.parentseg().sec]: #if parent section
#             self.pre_seg = [self.section((1+((1-self.section.nseg)/self.section.nseg))/2)] #set pre_seg to last segment of section
#         if len(set(self.pre_seg)) == 1 and len(self.pre_seg)>1:
#             self.pre_seg = [self.pre_seg[0]]
#         if single_seg:
#             self.init_seg = [self.init_seg[0]]
#         self.pre_v = Recorder(self.pre_seg)
#         self.post_v = Recorder(self.init_seg)
    
#     def get_current(self) -> np.ndarray:
#         v_pre = self.pre_v.as_numpy()
#         v_post = self.post_v.as_numpy()
#         axial_r = np.array([[seg.ri()] for seg in self.init_seg])
#         if (self.section.parentseg() is not None):
#           if self.init_sec == [self.section.parentseg().sec]: #in the case of section connected to parent section:
#             axial_r= np.array([[seg.ri()] for seg in self.pre_seg]) #ri() calculates from seg to parentseg() so adjust to go from pre_seg to init_seg rather than init_seg to pre_seg
#         axial_current = (v_pre-v_post)/axial_r
#         return axial_current

## Create simulation

In [11]:
x = np.zeros(96)
y = np.linspace(-1900,1900,96)
z = np.zeros(96)
elec_pos = pos = np.column_stack((x,y,z))

Set parameters

In [12]:
#choose orientation of neuron
# loc_param = [50,0,10,0,1,0]
loc_param = [50,0,10,0,1,0]  # example position #D: initially this was [0,0,50,0.2,0.75,1.5], (x,y,z,theta,h,phi), h=[-1,1] phi=[-pi,pi] h=.7071067812 for 90 deg
#note on parameters: Rotate(alpha,h,phi): first rotate alpha about y-axis (spin), then rotate arccos(h) about x-axis (elevation), then rotate phi about y axis (azimuth).


# change geo_parameters from their default values #************************NOTE**********************: doesn't change the geometry values used in geom_iterrows & sec['parameter']
                                                                                                       #(mainly when synapses are added using geom_parameters.csv);
geo_param = [ # -1 uses default value                                                                  #better to edit the geom file to guarantee edits until the automation is fixed
            #soma
            -1  # radius
        ]
      
      
# example of changing biophysical parameters (check "biophys_entries" in "Cell" class for meaning, bounds can also be found there)
#latest conductances from Detailed Model Updated 8/21/22
#Ih may be off for apical sections; The detailed model axon shas only g_pas, but we have given the axon some conductances closer to the soma, and may need to change
#also setting a conductance to 0.0 may throw some sort of error
biophys = [0.0000338,0.0000467,0.0000589,0.0000589,0.0000589,0.0000325,0.0000467, # g_pas of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #passive basal was 0.00001
                                    2.04,0.0213,0.0213,0.0213,0.0213,0.0,0.0, # gNaTa_t of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal # axon was 2.89618
                                    0.693,0.000261,0.000261,0.000261,0.000261,0.0,0.0, # gSKv3_1 of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #axon was 0.473799
           
                                    0.000992,0.0,0.0000555,(0.000555),.0000555,0.0,0.0,  # gCA_HVA of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #basal was 0.000992
                                    0.00343,0.0,0.000187,(0.0187),0.000187,0.0,0.0, # gCA_LVAst of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.0000675,0.0000675,(0.0000675),0.0000675,0.0, # gIm of basal, midTrunk, distTrunk, tuft, axon
           
                                    460.0,122,122,122,122,277.300774,122, # decay_CaDynamics of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.000501,0.000509,0.000509,0.000509,0.000509,0.000525,0.000509, # gamma_CaDynamics of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    # 0.0002,0.0002,0.0002,0.0002,0.0002,0.0001,0.0002, # gIh of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal
                                    0.0002,0.0002,0.0009596,0.0029005,.006997745,0.0,0.0, # gIh of soma, basal, midTrunk, distTrunk, tuft, axon, passiveBasal #calculated from distribute_channels #if sectioning tuft do disttuft:0.0118268; midtuft:.006997745; proxtuft:.0047005526
                                    0.0441,0.0012,0.0012,0.0012,0.0012,0.0, # gSk_E2 of soma, basal, midTrunk, distTrunk, tuft, axon
                                    0.0812,0.0, # gK_Tst of soma, axon #axon was 0.077274
                                    0.00223,0.0, # gK_Pst of soma, axon #axon was 0.188851
                                    0.00172,0.0  # gNap_Et2 of soma, axon
                                    ]
timestart=time.time()
sim = Simulation(geo_standard,elec_pos,loc_param,geo_param=geo_param,biophys=biophys,gmax=0.05,scale=100.,spike_threshold = 10, synaptic_input = True, RunType = 'Full Model')  # 0.001 -0.012 #gmax=.005, scale=100
timestop=time.time()
elapseddeftime=timestop-timestart
simtime=tstop/1000 #convert from ms to s
print('It took',round(elapseddeftime),'sec to define the simulation model.') #need to optomize input grouping # location check takes a long time since it iterates through checking a list of synapse locations for each group/cluster within the section

now attempting sec_id: 1
allsections: [soma[0]] | pid[i]: 0 | psec: soma[0]
allsections: [soma[0], proxbasal[0].dend[0]] | pid[i]: 0 | psec: soma[0]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1]] | pid[i]: 0 | psec: soma[0]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2]] | pid[i]: 0 | psec: soma[0]
now attempting sec_id: 2
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxbasal[3].dend[3]] | pid[i]: 1 | psec: proxbasal[0].dend[0]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxbasal[3].dend[3], midbasal[0].dend[4]] | pid[i]: 2 | psec: proxbasal[1].dend[1]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxbasal[3].dend[3], midbasal[0].dend[4], midbasal[1].dend[5]] | pid[i]: 3 | psec: proxbasal[2].dend[2]
allsections: [soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxba

In [13]:
sim.cells[0].all=sim.cells[0].allsections #change neuron SectionList object back to iterable List object

In [14]:
axc=Record_Axial_Current(sim.cells[0].all[0])

In [15]:
# def printRaster(raster, section, type):

#   title=str(section)+str(type)
#   nTrace = len(raster)

#   t = np.arange(0,tstop,0.1)
#     # t is an array with values ranging from 0 to t_stop with increment 0.1
#   z = np.zeros((nTrace,t.shape[0]))
#     # Z is a nTrace by t.shape[0] matrix of 0's

#   for i in np.arange(0,z.shape[0]):
#       plt.plot(raster[i],np.ones((raster[i].shape[0]))*i,'k.')
#   t = np.arange(0,tstop,0.1)

#   plt.xlabel('time(ms)')
#   plt.ylabel('node ID')
#   plt.title(title)
#   plt.show()

# #printRaster(ExcSpikeMatrix,'Exc Spike Trains')
# #printRaster(InhSpikeMatrix,'Inh Spike Trains')

In [16]:
# for i in range(len(sectioninhspikes)):
#   printRaster(sectioninhspikes[i],sim.cells[0].all[i],' Inh Spike Trains')

In [17]:
# for i in range(len(sectionexcspikes)):
#   printRaster(sectionexcspikes[i],sim.cells[0].all[i],' Exc Spike Trains')

##Track some model parameters

In [18]:
    # """A module for recording axial currents from nexus to segments attached to nexus"""
    # def __init__(self, disttrunk: nrn.Section, dend_type: Optional[str] = None, record_t: bool = False, single_seg: bool = False) -> None:
    #     """
    #     disttrunk: disttrunk section object
    #     dend_type: list of section names of the dendrite types that need to be recorded
    #     record_t: whether or not to record time points
#     #     single_seg: whether or not to record only one segment for each dendrite type
#     #     """
# # dir(axial_current)
# ac_idx = [0,-1] #not exactly sure what this does, but object cannot be enumerated otherwise
# # Nexus_axial_current = [Record_Axial_Current(sim.cells[0].all[19],single_seg=True)]
# # MidTrunk_axial_current = [Record_Axial_Current(sim.cells[i].all[18],single_seg=True) for i in ac_idx]
# Soma_axial_current = [Record_Axial_Current(sim.cells[0].all[0],single_seg=True)]

In [19]:
# axialcurrents=[]
# for i in range(len(sim.cells[0].all)):
#   axialcurrents.append([Record_Axial_Current(sim.cells[0].all[i],single_seg=True) for i in ac_idx])

In [20]:
# print(sim.cells[0].all[18]) #use to find section object for recording axial current

In [21]:
SecD_list=[]
for id,sec in sim.cells[0].geometry.iterrows():
  sec_index_list=sim.cells[0].sec_id_lookup[id]
  D = sec['R']*2
  if sec['axial']:
    SecD_list.append(D)
  else:
    SecD_list.append(D)
    SecD_list.append(D)
    SecD_list.append(D)
    SecD_list.append(D)
# print(SecD_list)

SecL_list=[]
for id,sec in sim.cells[0].geometry.iterrows():
  sec_index_list=sim.cells[0].sec_id_lookup[id]
  L = sec['L']
  if sec['axial']:
    SecL_list.append(L)
  else:
    SecL_list.append(L)
    SecL_list.append(L)
    SecL_list.append(L)
    SecL_list.append(L)
# print(SecL_list)

Record membrane voltage at initial/middle/end of the trunk and middle of the tuft to check the propagation.

In [22]:
nseg = len(sim.cells[0].segments)
# print(nseg)
numTstep = int(h.tstop/h.dt)

SegX = []

seg_list = sim.cells[0].get_seg_by_id(range(nseg))
#print(seg_list)

Vrecord = []
for i in range(len(seg_list)):
     Vrecord.append(seg_list[i])
     SegX.append(seg_list[i].x)

dend_v = Recorder(Vrecord)

sim.cells[0]

In [23]:
def returnSegmentValues(section, secnum):
    ##
    
    #Section naming
    #Set impedance measurement location and frequency
   
    beta_zz = h.Impedance()
    beta_zz.loc(sim.cells[0].get_sec_by_id([0])[0](0.5))
    beta_zz.compute(24) #inj frequency, 0, beta: 20 or 24, gamma: 64 hz
    gamma_zz = h.Impedance()
    gamma_zz.loc(sim.cells[0].get_sec_by_id([0])[0](0.5))
    gamma_zz.compute(64) #inj frequency, 0, beta: 20 or 24, gamma: 64 hz
    

    # Get section 3d coordinates and put in numpy array
    n3d = section.n3d()
    x3d = np.empty(n3d)
    y3d = np.empty(n3d)
    z3d = np.empty(n3d)
    L = np.empty(n3d)
    for i in range(n3d):
        x3d[i]=section.x3d(i)
        y3d[i]=section.y3d(i)
        z3d[i]=section.z3d(i)

    # Compute length of each 3d segment
    for i in range(n3d):
        if i==0:
            L[i]=0
        else:
            L[i]=np.sqrt((x3d[i]-x3d[i-1])**2 + (y3d[i]-y3d[i-1])**2 + (z3d[i]-z3d[i-1])**2)

    # Get cumulative length of 3d segments
    cumLength = np.cumsum(L)
    N = section.nseg
    # Now upsample coordinates to segment locations
    xCoord = np.empty(N)
    yCoord = np.empty(N)
    zCoord = np.empty(N)
    B_Elec_distance = np.empty(N)
    G_Elec_distance = np.empty(N)
    SecNum = np.empty(N)
    SecName = []
    dist = np.empty(N)
    Type = []
    segdiams = np.empty(N)
    X = np.empty(N)
    apic = ['proxtrunk','oblique','midtrunk','disttrunk','proxtuft','midtuft','disttuft']
    dend = ['soma','proxbasal','midbasal','distbasal','passivebasal','axon']  


    if N > 1:
      dx = section.L / (N-1)
    else:
      dx = section.L

    #print(str(section))

    if str(section) in apic:
      #print(str(section) + ' is apic')
      for n in range(N):
        Type.append('apic')
        
    else: 
      #print(str(section) + ' is dend')
      for n in range(N):
        Type.append('dend')
        

    for n in range(N):

      SecName.append('L5PCtemplate[0].'+ str(Type[0]) + '[' + str(secnum) + ']')

      SecNum[n] = secnum
      B_Elec_distance[n] = beta_zz.ratio(section((n*dx)/cumLength[1]))
      G_Elec_distance[n] = gamma_zz.ratio(section((n*dx)/cumLength[1]))
      dist[n] = h.distance(sim.cells[0].get_sec_by_id([0])[0](0.5), section((n*dx)/cumLength[1]))
      segdiams[n] = h.distance(sim.cells[0].get_sec_by_id([0])[0](0.5), section((n*dx)/cumLength[1]))

      if n==(N-1): 
          xCoord[n]=x3d[-1]
          yCoord[n]=y3d[-1]
          zCoord[n]=z3d[-1]
      else:
          cIdxStart = np.where(n*dx >= cumLength)[0][-1] # which idx of 3d segments are we starting at
          cDistFrom3dStart = n*dx - cumLength[cIdxStart] # how far along that segment is this upsampled coordinate
          cFraction3dLength = cDistFrom3dStart / L[cIdxStart+1] # what's the fractional distance along this 3d segment
          # compute x and y positions
          xCoord[n] = x3d[cIdxStart] + cFraction3dLength*(x3d[cIdxStart+1] - x3d[cIdxStart])
          yCoord[n] = y3d[cIdxStart] + cFraction3dLength*(y3d[cIdxStart+1] - y3d[cIdxStart])
          zCoord[n] = z3d[cIdxStart] + cFraction3dLength*(z3d[cIdxStart+1] - z3d[cIdxStart])
            
    return SecNum, xCoord, yCoord, zCoord, B_Elec_distance, dist, Type, SecName, L, G_Elec_distance

In [24]:
sec_list = sim.cells[0].get_sec_by_id(range(len(sim.cells[0].all)))
print(sec_list)
# print(nexus_segdist)
# sim.cells[0].segments[nexus_seg]

[soma[0], proxbasal[0].dend[0], proxbasal[1].dend[1], proxbasal[2].dend[2], proxbasal[3].dend[3], midbasal[0].dend[4], midbasal[1].dend[5], midbasal[2].dend[6], midbasal[3].dend[7], distbasal[0].dend[8], distbasal[1].dend[9], distbasal[2].dend[10], distbasal[3].dend[11], proxtrunk[0].apic[0], oblique[0].apic[1], oblique[1].apic[2], oblique[2].apic[3], oblique[3].apic[4], midtrunk[0].apic[5], disttrunk[0].apic[6], proxtuft[0].apic[7], proxtuft[1].apic[8], proxtuft[2].apic[9], proxtuft[3].apic[10], midtuft[0].apic[11], midtuft[1].apic[12], midtuft[2].apic[13], midtuft[3].apic[14], disttuft[0].apic[15], disttuft[1].apic[16], disttuft[2].apic[17], disttuft[3].apic[18], axon[0].axon[0], passivebasal[1], passivebasal[2], passivebasal[3], passivebasal[4]]


In [25]:
seccolors=['grey', #soma
        'b','b','b','b', #proxbasal
        'c','c','c','c', #midbasal
        'g','g','g','g', #distbasal
        'purple', #proxtrunk
        'purple','purple','purple','purple', #oblique
        'm', #midtrunk
        'c', #disttrunk
        'y','y','y','y', #proxtuft
        'orange','orange','orange','orange', #midtuft
        'r','r','r','r', #disttuft
        'black', #axon
        'purple','purple','purple','purple' #passivebasal
        ]

In [26]:
sec_list = sim.cells[0].get_sec_by_id(range(len(sim.cells[0].all)))

section = []
SectionXCoord = []
SectionYCoord = []
SectionZCoord = []
SectionElecDist = []
SectionDist = []
SectionType = []
SectionName = []

AllSegSection = []
AllSegXCoord = []
AllSegYCoord = []
AllSegZCoord = []
AllSegBElecDist = []
AllSegGElecDist = []
AllSegDist = []
AllSegType = []
AllSegSecName = []
AllSegLength = []

probes=[]

class probe():
  def __init__(self,seg,sectionindex,sectionname,color,excspikes,inhspikes,section):
    self.seg=seg
    self.sectionindex=sectionindex
    self.sectionname=sectionname
    self.color=color
    self.excsecspikes=excspikes
    self.inhsecspikes=inhspikes
    try:
      self.axial_current=[Record_Axial_Current(section,single_seg=True)]
    except:
      print(section)

for i in range(len(sec_list)):

  section, SectionXCoord, SectionYCoord, SectionZCoord, SectionBElecDist, SectionDist, SectionType, SectionName, SegmentLength, SectionGElecDist = returnSegmentValues(sec_list[i], i)

  AllSegSection.extend(section)
  AllSegXCoord.extend(SectionXCoord)
  AllSegYCoord.extend(SectionYCoord)
  AllSegZCoord.extend(SectionZCoord)
  AllSegBElecDist.extend(SectionBElecDist)
  AllSegGElecDist.extend(SectionGElecDist)
  AllSegDist.extend(SectionDist)
  AllSegType.extend(SectionType)
  #print(AllSegType)
  AllSegSecName.extend(SectionName)
  AllSegLength.extend(SegmentLength)
  try:
    seg = round(len(AllSegSection) - (len(section)/2))
  except:
    seg = 0
  newprobe=probe(seg=seg,sectionindex=i,sectionname=str(sec_list[i]),color=seccolors[i],excspikes=sectionexcspikes[i],inhspikes=sectioninhspikes[i],section=sim.cells[0].all[i])
  probes.append(newprobe)

In [27]:
print(len(sectioninhspikes))
print(len(sectionexcspikes))

37
37


In [28]:
gNaTa_T = Recorder(obj_list = sim.cells[0].segments, var_name = 'gNaTa_t_NaTa_t')
ina = Recorder(obj_list = sim.cells[0].segments, var_name = 'ina_NaTa_t')
ical = Recorder(obj_list = sim.cells[0].segments, var_name = 'ica_Ca_LVAst')
icah = Recorder(obj_list = sim.cells[0].segments, var_name = 'ica_Ca_HVA')
ih = Recorder(obj_list = sim.cells[0].segments, var_name = 'ihcn_Ih')

In [29]:
# im = Recorder(obj_list = sim.cells[0].segments, var_name = 'ik_Im')

##Run the simulation

In [30]:
timestart=time.time()
sim.run()
timestop=time.time()
lfp = sim.get_lfp().T
t = sim.t()
elapsedtime=timestop-timestart
simtime=tstop/1000 #convert from ms to s
totaltime= elapsedtime+elapseddeftime
print('It took',round(elapsedtime),'sec to run a',simtime,'sec simulation.')
print('Inflation Factor:',elapsedtime/simtime)
print('The total runtime was',round(totaltime),'sec')

It took 40 sec to run a 1.0 sec simulation.
Inflation Factor: 40.172626972198486
The total runtime was 47 sec


In [31]:
#compare to detailed model (detail model values subject to change)
detailed_syns=29028
detailed_segs=2526
print('The number of synapses in the detailed model is ',detailed_syns)
print('The number of synapses in the reduced-order model is',len(sim.cells[0].injection))
try: syn_reduce = (len(sim.cells[0].injection)/detailed_syns)
except: syn_reduce = 0
print('The number of synapses has been reduced by',1-syn_reduce,'%')
print()
print('The number of segments in the detailed model is ',detailed_segs)
print('The number of segments in the reduced-order model is',len(sim.cells[0].segments))
seg_reduce = (len(sim.cells[0].segments)/detailed_segs)
print('The number of segments has been reduced by a factor of',(1-seg_reduce),'%')

# Ben's detailed L5
# Excitatory: 16070 on apical dendrites (2.16 syns/um * 7440 um)
# 10042 (2.16 syns/um * 4649 um) on basal dendrites. 
# Inhibitory: 150 on soma  (Egger et al., 2020; Markram et al., 2015; Reimann et al., 2015) + (0.22 syns/um *483 um) = 256 perisomatic, 
# 1637 (0.22 syns/um * 7440 um)  on apical dendrites, 
# 1023 (0.22 syns/um * 4649 um) on basal dendrites.

# total is 29028

#note: 
# Ben's apical to basal length & apical to basal synapse ratio is 1.6003 (7440um/4649um)

The number of synapses in the detailed model is  29028
The number of synapses in the reduced-order model is 4020
The number of synapses has been reduced by 0.8615130219098801 %

The number of segments in the detailed model is  2526
The number of segments in the reduced-order model is 895
The number of segments has been reduced by a factor of 0.6456848772763262 %


##Retrieve Data

In [32]:
numSyn = len(sim.cells[0].injection)

excSynPerSeg = [0]*nseg
inhSynPerSeg = [0]*nseg
excSynPerSegL = [0]*nseg
inhSynPerSegL = [0]*nseg
SynParentSeg = []
SourcePop = []
SynType = []
SynDist = []

i_NMDA_bySeg= [[0] * (numTstep+1) ] * nseg

#print(len(sim.cells[0].injection))

for j in range(numSyn):
  seg = sim.cells[0].injection[j].get_segment_id() 
  SynParentSeg.append(seg)
  SynType.append(AllSegType[seg])
  SynDist.append(AllSegDist[seg])

  if(sim.cells[0].injection[j].syntype == 'exc'):
    excSynPerSeg[seg] += 1
    SourcePop.append('exc_stim')
  else:
    inhSynPerSeg[seg] += 1
    SourcePop.append('dist_inh_stim')

  try:
    i_NMDA = np.array(sim.cells[0].injection[j].rec_vec.vec_list[1])            #current = numpy array of NEURON Vector of current NMDA current at synapse j  
    seg = sim.cells[0].injection[j].get_segment_id()                            #seg = the segment in which synapse j is located 
    #print('first try')
    
    try:
      i_NMDA_bySeg[seg] = i_NMDA_bySeg[seg] + i_NMDA    
      #print('second try')                                           #Sum current over each segment
    except: 
      x = j                                                                     #Except needed as some synpases do not have NMDA currrent and throw an error when called
  except:
    x = j

In [33]:
len(sim.cells[0].injection)
len(AllSegType)

895

In [34]:
# print(dir(sim.cells[0].injection[0]))

syn_dist_list = []
gmax_list = []
P_0_list = []
synapse_sec_list = []
SynName = []

for i in range(len(sim.cells[0].injection)):
  syn_dist_list.append(sim.cells[0].injection[i].loc)
  gmax_list.append(sim.cells[0].injection[i].gmax)
  P_0_list.append(sim.cells[0].injection[i].P_0)
  synapse_sec_list.append(sim.cells[0].injection[i].sec_index)

  SynName.append('L5PCtemplate[0].'+ str(SynType[i]) +'[' + str(synapse_sec_list[i]) + '](' + str(syn_dist_list[i]) + ')')


#L5PCtemplate[0].dend[60](0.264706)
# print(len(syn_dist_list))
# print(len(gmax_list))
# print(len(P_0_list))
# print(len(synapse_sec_list))
# print(len(SynName))

In [35]:
ih_data = ih.as_numpy()
gNaTa_T_data = gNaTa_T.as_numpy()
ina_data = ina.as_numpy()
icah_data = icah.as_numpy()
ical_data = ical.as_numpy()
v_dend = dend_v.as_numpy()
# im_data = im.as_numpy()

In [36]:
spikes = sim.get_spike_time()

#spikes = [500]
print(len(spikes)* (1000/tstop))

10.0


# File Output

In [37]:
def createsegtracereport(reportname,dataname):
  try:
    os.remove(reportname) # reportname was string " "
  except:
    x = 1

  f = h5py.File(reportname,'w') #create a file in the w (write) mode #reportname was string ' '
  v = f.create_dataset("report/biophysical/data", data = dataname)
  f.close()

In [38]:
segID = list(range(nseg))

In [39]:
# Check segment parameters to debug

# print(len(segID))
# print(len(SegX))
# print(len(AllSegType))
# print(len(AllSegSection))
# print(len(AllSegDist))
# print(len(AllSegXCoord))
# print(len(AllSegYCoord))

In [40]:
#Segments.csv

try:
  os.remove("L5Segments.csv")
except:
  print('No L5Segments.csv to remove.')

# BMTK ID: segment ID
# X: Normalized distance of segment along parent section 
# Type: apic, basal, soma 
# Sec ID: section ID of segment's parent section
# Distance: path distance of segment from soma
# Coord X/Y/Z: 3D coordinates 
# Elec_distance: Elcectrotonic distance

#segments_data = {'BMTK ID':[segID],'X':[SegX],'Type':[AllSegType],'Sec ID':[AllSegSection],'Distance':[AllSegDist],'Coord X':[AllSegXCoord],'Coord Y':[AllSegYCoord],'Coord Z':[AllSegZCoord],'Elec_distance':[AllSegElecDist]}
#segments = pd.DataFrame(segments_data, columns = ['BMTK ID','X','Type','Sec ID','Distance','Coord X','Coord Y','Coord Z','Elec_distance'])


segments = pd.DataFrame({'segmentID':segID,'X':SegX,'Type':AllSegType,'Sec ID':AllSegSection,'Distance':AllSegDist,'Coord X':AllSegXCoord,'Coord Y':AllSegYCoord,'Coord Z':AllSegZCoord,'Elec_distance':AllSegGElecDist})

segments.to_csv('L5Segments.csv', index = False)

No L5Segments.csv to remove.


In [41]:
# def save_degrees(cell):
#     degrees = {}
#     calculate_degree(h.SectionRef(sec=cell.soma), degrees, 0)

#     df_dict = {}
#     df_dict["SectionName"] = list(degrees.keys())
#     df_dict["Degrees"] = list(degrees.values())
    
#     df = pd.DataFrame(df_dict)
#     try:
#       os.remove("ReducedSegmentDegrees.csv")
#     except:
#       print('No ReducedSegmentDegrees.csv to remove.')
#     df.to_csv("ReducedSegmentDegrees.csv", index=False)

# def calculate_degree(sref, degrees, deg):
#     degrees[sref.sec.name()] = deg

#     for c in sref.child:
#         calculate_degree(h.SectionRef(sec=c), degrees, deg+1)

# def make_seg_df(cell):
#     secdegrees=pd.read_csv('ReducedSegmentDegrees.csv')
#     df = pd.DataFrame()
#     i = 0
#     j = 0
#     sec_types_bysec=[]
#     lens = []
#     seg_lens = []
#     diams = []
#     segdiams = []
#     bmtk_ids = []
#     seg_ids = []
#     sec_ids = []
#     full_names = []
#     xs = []
#     parts = []
#     distances = []
#     elec_distances = []
#     elec_distances_nexus = []
#     h.distance(sec=cell.all[0])
#     nsegs=[]
#     RAs=[]
#     Parentx=[]
#     zz = h.Impedance()
#     zz.loc(cell.all[0](0.5))
#     zz.compute(25)
#     ww = h.Impedance()
#     ww.loc(cell.all[19](0.5)) #disttrunk
#     ww.compute(25)
#     psegids=[]
#     segments=[]
#     sections=[]
#     fullsecnames=[]
#     seg_degrees=[]
#     apic = ['proxtrunk','midtrunk','disttrunk','proxtuft','midtuft','disttuft','oblique']
#     dend = ['proxbasal','midbasal','distbasal']

#     for sec in cell.all:
#         print(sec.name)
#         for seg in sec:

#             lens.append(seg.sec.L)
#             seg_lens.append(seg.sec.L/seg.sec.nseg)
#             diams.append(seg.sec.diam)
#             segdiams.append(seg.diam)
#             distances.append(h.distance(seg))
#             bmtk_ids.append(i)
#             seg_ids.append(j)
#             xs.append(seg.x)
#             fullsecname = sec.name()
#             fullsecnames.append(fullsecname)
#             degree=secdegrees.loc[secdegrees.SectionName==fullsecname]['Degrees']
#             seg_degrees.append(int(degree))
#             if str(fullsecname)=='soma':
#               sec_type = 'soma'
#             elif str(fullsecname) in apic:
#               sec_type = 'apic'
#             elif str(fullsecname) in dend:
#               sec_type = 'dend'
#             else:
#               sec_type = str(fullsecname)
#             sec_id=sec_types_bysec.count(sec_type)
#             sec_ids.append(sec_id)
#             nsegs.append(seg.sec.nseg)
#             RAs.append(seg.sec.Ra)
#             parts.append(sec_type)
#             full_names.append(str(seg))
#             elec_distances.append(zz.ratio(seg))
#             elec_distances_nexus.append(ww.ratio(seg))
#             #if seg.sec.parentseg() is not None:
#               #get parentseg coordinates or something to identify parentseg by
#             j += 1
#             segments.append(seg)
#         sec_types_bysec.append(sec_type)
#         i += 1
#         sections.append(sec)
#     #print(segments)
#     for i in range(len(segments)): #calculate parentseg id using seg index on section
#       idx = int(np.floor(segments[i].x * segments[i].sec.nseg)) #get seg index on section
#       #case where segment is not first segment on section:
#       if idx != 0: #if the segment is not the first on the section then the parent segment is the previous segment index
#         psegid=i-1 #set parent segment id to the previous segment index
#         psegids.append(psegid)
#       #case where segment is first segment on section:
#       else:
#         pseg = segments[i].sec.parentseg()
#         if pseg == None:
#           psegids.append(None)
#         else:
#           psec=pseg.sec
#           nseg = psec.nseg
#           pidx = int(np.floor(pseg.x * nseg)) #get parent seg index on section
#           if pseg.x == 1.:
#             pidx -= 1
#           psegid=segments.index(psec((pidx + .5) / nseg)) #find the segment id of the parent seg by comparing with segment list after calculating the parent seg's section(x)
#           psegids.append(psegid)


    # numSyn = len(cell.injection)
    # nseg = len(sim.cells[0].segments)
    # excSynPerSeg = [0]*nseg
    # inhSynPerSeg = [0]*nseg
    # excSynPerSegL = [0]*nseg
    # inhSynPerSegL = [0]*nseg
    # SynParentSeg = []
    # SourcePop = []
    # SynType = []
    # SynDist = []

    # i_NMDA_bySeg= [[0] * (numTstep+1) ] * nseg

    # #print(len(sim.cells[0].injection))

    # for j in range(numSyn):
    #   seg = cell.injection[j].get_segment_id() 
    #   SynParentSeg.append(seg)
    #   SynType.append(AllSegType[seg])
    #   SynDist.append(AllSegDist[seg])

    #   if(cell.injection[j].syntype == 'exc'):
    #     excSynPerSeg[seg] += 1
    #     SourcePop.append('exc_stim')
    #   else:
    #     inhSynPerSeg[seg] += 1
    #     SourcePop.append('dist_inh_stim')
#     df["segmentID"] = seg_ids
#     df["BMTK ID"] = bmtk_ids
#     df["Seg_L"] = seg_lens
#     df["Seg_diam"] = segdiams
#     df["X"] = xs
#     df["Type"] = parts
#     df["Sec ID"] = sec_ids
#     df["Distance"] = distances
#     df["Section_L"] = lens
#     df["Section_diam"] = diams
#     df["Section_nseg"] = nsegs
#     df["Section_Ra"] = RAs
#     df["Coord X"] = AllSegXCoord
#     df["Coord Y"] = AllSegYCoord
#     df["Coord Z"] = AllSegZCoord
#     df["ParentSegID"] = psegids
#     df["Elec_distance"] = elec_distances
#     df["Elec_distance_nexus"] = elec_distances
#     df["Sec Name"] = fullsecnames
#     df['Degrees']=seg_degrees
#     df['num_syns_exc']=excSynPerSeg
#     df['num_syns_inh']=inhSynPerSeg


#     df.to_csv("ReducedSegments.csv", index=False)
# try:
#   os.remove("ReducedSegments.csv")
# except:
#   print('No ReducedL5Segments.csv to remove.')

try:
  os.remove("ReducedSegments.csv")
except:
  print('No ReducedL5Segments.csv to remove.')

save_degrees(sim.cells[0])
make_seg_df(sim.cells[0])

No ReducedL5Segments.csv to remove.
No ReducedSegmentDegrees.csv to remove.
<built-in method name of nrn.Section object at 0x7fb8164ff1b0>
count(sec_type): 0
<built-in method name of nrn.Section object at 0x7fb8164ff0c0>
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
<built-in method name of nrn.Section object at 0x7fb8164ff5d0>
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
<built-in method name of nrn.Section object at 0x7fb8164ffa80>
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
<built-in method name of nrn.Section object at 0x7fb8164fff30>
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
count(sec_type): 0
<built-in method name of nrn.Section object at 0x7fb816508

NameError: ignored

In [ ]:
secdegrees=pd.read_csv('ReducedSegmentDegrees.csv')
degree=secdegrees.loc[secdegrees.SectionName=='proxtuft']['Degrees']
# print(int(degree))

In [ ]:
import matplotlib

In [ ]:
segs = pd.read_csv('ReducedSegments.csv')
plt.figure(figsize=(4,10))
ax = plt.scatter(segs['Coord X'], segs['Coord Y'],c=100*segs['Elec_distance'],s=4,cmap='jet',norm=matplotlib.colors.LogNorm())
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
cbar = plt.colorbar()
cbar.ax.set_ylabel('Percentage of Somatic signal', rotation=270)

#ax2.ax.set_title('log(elec_distance)',rotation=270)
plt.box(False)
plt.savefig('Elec_distance.svg')

In [ ]:
Degrees = [0] * (nseg-1)

In [ ]:
#SegmentsDegrees

try:
  os.remove("SegmentsDegrees.csv")
except:
  x = 1

#segmentsDegrees_data = {'SectionName':[AllSegSecName],'BMTK ID':[segID],'X':[SegX],'Type':[AllSegType],'Sec ID':[AllSegSection],'Distance':[AllSegDist],'Coord X':[AllSegXCoord],'Coord Y':[AllSegYCoord],'Coord Z':[AllSegZCoord],'Degrees':[Degrees]}
#segmentsDegrees = pd.DataFrame(segmentsDegrees_data, columns = ['SectionName','BMTK ID','X','Type','Sec ID','Distance','Coord X','Coord Y','Coord Z','Degrees'])

# print(len(AllSegSecName))
# print(len(segID))
# print(len(SegX))
# print(len(AllSegType))
# print(len(AllSegDist))
# print(len(AllSegXCoord))
# print(len(AllSegYCoord))
# print(len(AllSegZCoord))
# print(len(Degrees))


segmentsDegrees = pd.DataFrame({'SectionName': AllSegSecName,'BMTK ID': segID ,'X': SegX ,'Type': AllSegType ,'Sec ID': AllSegSection ,'Distance': AllSegDist ,'Coord X': AllSegXCoord ,'Coord Y': AllSegYCoord,'Coord Z': AllSegZCoord,'Degrees': segID })

segmentsDegrees.to_csv('SegmentsDegrees.csv', index = False)

In [ ]:
#create new output folder
output_folder='output_control'+str(tstop)
cwd = os.getcwd() #get current directory
target_dir = cwd +'/'+output_folder # desired forward name + current directory for new folder location and name
if os.path.exists(target_dir)==False: #if the directory does not exist
    os.mkdir(target_dir)

In [ ]:
# %cd output_control


In [ ]:
#Connections.csv


try:
  os.remove('/'+output_folder+'/Connections.csv')
except:
  x = 1

# Node ID: Segment ID of synapse's parent segment
# Distance: Path distance from soma (using same dist as parent segment)
# Conductance: G_max of syn
# Type: apic, basal, soma
# Name: L5PCtemplate[0].(Type)[(Sec ID)]((Normalized distance along section))
# Source Population: exititory or inhibitory
# Release Probability: Release porbablity at time 0 

# connections_data = {'Node ID':[SynParentSeg],'Distance':[SynDist],'Conductance':[gmax_list],'Type':[SynType],'Name':[SynName],'Source Population':[SourcePop],'Release Probability':[P_0_list]}

# # Create the pandas DataFrame with column name is provided explicitly
# connections = pd.DataFrame(connections_data , columns=['Node ID','Distance','Conductance','Type','Name','Source Population','Release Probability'])

connections = pd.DataFrame({'Node ID':SynParentSeg,'Distance':SynDist,'Conductance':gmax_list,'Type':SynType,'Name':SynName,'Source Population':SourcePop,'Release Probability':P_0_list})

connections.to_csv(output_folder+'/Connections.csv', index = False)


In [ ]:
#print(i_NMDA_bySeg)
i_NMDA_df = pd.DataFrame(i_NMDA_bySeg) * 1000


In [ ]:
createsegtracereport(output_folder+'/v_report.h5', v_dend.T)
createsegtracereport(output_folder+'/Ca_HVA.ica_report.h5',icah_data.T)
createsegtracereport(output_folder+'/Ca_LVAst.ica_report.h5',ical_data.T)
createsegtracereport(output_folder+'/Ih.ihcn_report.h5',ih_data.T)
createsegtracereport(output_folder+'/inmda_report.h5',i_NMDA_df.T)
createsegtracereport(output_folder+'/NaTa_t.gNaTa_t_report.h5',gNaTa_T_data.T)
createsegtracereport(output_folder+'/NaTa_t.ina_report.h5',ina_data.T)

In [ ]:
try:
  os.remove(output_folder+"spikes.h5")
except:
  x = 1

f = h5py.File(output_folder+'/spikes.h5','w') #create a file in the w (write) mode
v = f.create_dataset("spikes/biophysical/timestamps", data = spikes)

f.close()

In [ ]:
# %cd ..

In [ ]:
def plot_elec_dist(ElecDist,title,filename):
  plt.figure(figsize=(4,10))
  ax = plt.scatter(AllSegXCoord, AllSegYCoord, c = np.log(ElecDist) )
  plt.vlines(110,300,400)
  plt.text(0,350,'100 um')
  plt.hlines(300,110,210)
  plt.text(110,250,'100 um')
  plt.xticks([])
  plt.yticks([])
  plt.title(title)
  cbar = plt.colorbar()
  cbar.ax.set_ylabel('log(elec_distance)', rotation=270)
  plt.box(False)
  plt.savefig('Plots/SROL5Plots/'+filename)

# Plots

In [ ]:
# print(AllSegLength)

In [ ]:
# len(AllSegLength)

In [ ]:
# excSynPerUM=np.zeros(nseg)
# for seg in range(nseg-1):
#   excSynPerUM[seg]=excSynPerSeg[seg]/AllSegLength[seg]
# print(excSynPerUM)

In [ ]:
def plotSynDensity(SynPerSeg):
  plt.figure(figsize=(4,10))
  ax = plt.scatter(AllSegXCoord, AllSegYCoord,c = SynPerSeg[0:nseg],cmap='jet',)
  plt.vlines(110,400,500)
  plt.text(0,450,'100 um')
  plt.hlines(400,110,210)
  plt.text(110,350,'100 um')
  plt.xticks([])
  plt.yticks([])
  cbar = plt.colorbar()
  cbar.ax.set_ylabel('Synapses per segment', rotation=270)

  plt.box(False)
  plt.savefig('Plots/SROL5Plots/exc_syns.svg')

def plotSynDensityNoSoma(SynPerSeg):
  plt.figure(figsize=(4,10))
  ax = plt.scatter(AllSegXCoord[1:], AllSegYCoord[1:],c = SynPerSeg[1:nseg],cmap='jet',)
  plt.vlines(110,400,500)
  plt.text(0,450,'100 um')
  plt.hlines(400,110,210)
  plt.text(110,350,'100 um')
  plt.xticks([])
  plt.yticks([])
  cbar = plt.colorbar()
  cbar.ax.set_ylabel('Synapses per segment', rotation=270)

  plt.box(False)
  plt.savefig('Plots/SROL5Plots/inh_syns.svg')

In [ ]:
#for plotting morphology
seccolors = ['g',       #soma
'yellow',  #proxbasal
'orange',  #midbasal
'red',     #distbasal
'yellow',  #proxtrunk
'purple',  #oblique
'blue',    #midtrunk
'green',   #disttrunk
'yellow',  #proxtuft
'orange',  #midtuft
'red',     #disttuft
'c',       #axon
'black']  #passivebasal
# 'black' #passivetuft

In [ ]:
x_dist = np.unique(elec_pos[:, 0])  # unique x coordinates of electrode array (columns)
print(x_dist)

ix = 1  # select a column
y_window = [-1000, 1000]  # select range of y coordinate for plotting

Axial Current From Nexus to Tuft:

Try:
Subplot next to the voltage traces
axial current going other way

In [ ]:
def plot_axial_current(axial_current,title):
  dend_types = axial_current[0].dend_type
  plt.figure(figsize=(12.8, 4.8))
  for i,AC in enumerate(axial_current):
      ac = AC.get_current()
      for dend_type in dend_types:
          plt.plot(t,ac[dend_type].ravel(),label=dend_type)
      plt.ylabel('nA')
      plt.legend()
      plt.title(title)
      plt.xlabel('time (ms)')
  plt.show()

In [ ]:
# plot_axial_current(MidTrunk_axial_current,'Axial current from Midtrunk with synaptic input')

In [ ]:
# plot_axial_current(Nexus_axial_current,'Axial current from Nexus with synaptic input')

In [ ]:
# plot_axial_current(Soma_axial_current,'Axial current from Soma with synaptic input') # figure out how to plot these currents individually. & maybe plot the soma membrane voltage

##Voltage Traces

Try to subplot currents next to the voltage traces

In [ ]:
z=np.zeros((20,t.shape[0]))
len(z)

In [ ]:
def get_probe(probe):
  name=str(probe.sectionname)+str(probe.sectionindex)
  segment=probe.seg
  color=probe.color
  excsecspikes=probe.excsecspikes
  inhsecspikes=probe.inhsecspikes
  return name,segment,color,excsecspikes,inhsecspikes

def plot_spikes(spikes,title):
    nTrace = len(spikes)
    z = np.zeros((nTrace,t.shape[0])) # Z is a nTrace by t.shape[0] matrix of 0's
    for i in np.arange(0,z.shape[0]):
      plt.plot(spikes[i],np.ones((spikes[i].shape[0]))*i,'k.')
    plt.ylabel('node ID')
    plt.title(title)

def plot_trace(segment,color,title,data):
  plt.plot(np.arange(0,((h.tstop)+.1),.1),data[segment],color=color)
  plt.hlines(-60,0,tstop, color = 'grey', linestyle = 'dashed')
  plt.hlines(-40,0,tstop, color = 'grey', linestyle = 'dashed')
  plt.hlines(0,0,tstop, color = 'grey', linestyle = 'dashed')
  plt.text(2,3,'0 mV')
  plt.hlines(-10,50,70)
  plt.vlines(50,-10,0)
  plt.text(50,-15,'10 ms')
  plt.text(2,-5,'10 mV')
  plt.text(2,-37,'-40 mV')
  plt.text(2,-57,'-60 mV')
  plt.title(title)
  plt.box(False)
  plt.xticks([])
  plt.yticks([])

def plot_voltage_trace(probe,title=None,savename=None):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  if savename==None:
    savename=name+'_Voltage.png'
  if title==None:
    title=name+' Voltage'
  plt.figure(figsize=(15,4))
  plot_trace(segment,color,title,data=v_dend)
  plt.savefig('Plots/SROL5Plots/'+savename)

def plot_voltage_trace_lastsecond(probe,title=None,savename=None):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  if savename==None:
    savename=name+'_Voltage.png'
  if title==None:
    title=name+' Voltage'
  plt.figure(figsize=(15,4))
  v=v_dend[segment]
  plt.plot(np.arange(h.tstop-1000,(h.tstop+h.dt/2),h.dt),v[int((h.tstop-1000)/h.dt)::],color=color)
  plt.hlines(-60,tstop-1000,tstop, color = 'grey', linestyle = 'dashed')
  plt.hlines(-40,tstop-1000,tstop, color = 'grey', linestyle = 'dashed')
  plt.hlines(0,tstop-1000,tstop, color = 'grey', linestyle = 'dashed')
  plt.text(2+tstop-1000,3,'0 mV')
  plt.hlines(-10,50+tstop-1000,70+tstop-1000)
  plt.vlines(50+tstop-1000,-10,0)
  plt.text(50+tstop-1000,-15,'10 ms')
  plt.text(2+tstop-1000,-5,'10 mV')
  plt.text(2+tstop-1000,-37,'-40 mV')
  plt.text(2+tstop-1000,-57,'-60 mV')
  plt.title(title)
  plt.box(False)
  plt.xticks([])
  plt.yticks([])
  plt.savefig('Plots/SROL5Plots/'+savename)

  
def plot_voltage_trace_last500ms(probe,title=None,savename=None,color=None):
  if color==None:
    name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  else:
    name,segment,notcolor,excsecspikes,inhsecspikes=get_probe(probe)
  duration=500
  if savename==None:
    savename=name+'_Voltage.png'
  if title==None:
    title=name+' Voltage'
  plt.figure(figsize=(15,4))
  v=v_dend[segment]
  plt.plot(np.arange(h.tstop-duration,(h.tstop+h.dt/2),h.dt),v[int((h.tstop-duration)/h.dt)::],color=color)
  plt.hlines(-60,tstop-duration,tstop, color = 'grey', linestyle = 'dashed')
  plt.hlines(-40,tstop-duration,tstop, color = 'grey', linestyle = 'dashed')
  plt.hlines(0,tstop-duration,tstop, color = 'grey', linestyle = 'dashed')
  plt.text(2+tstop-duration,3,'0 mV')
  plt.hlines(-10,50+tstop-duration,150+tstop-duration)
  plt.vlines(50+tstop-duration,-10,0)
  plt.text(50+tstop-duration,-15,'100 ms')
  plt.text(2+tstop-duration,-5,'10 mV')
  plt.text(2+tstop-duration,-37,'-40 mV')
  plt.text(2+tstop-duration,-57,'-60 mV')
  plt.title(title)
  plt.box(False)
  plt.xticks([])
  plt.yticks([])
  plt.savefig('Plots/SROL5Plots/'+savename)

def subplot_voltagetrace_withspikes(probe):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  savename=name+'.png'
  plt.figure(figsize=(15,12))
  plt.subplot(3,1,1)
  title=name+' Voltage Trace'
  plot_trace(segment,color,title,data=v_dend[segment])

  t = np.arange(0,tstop,0.1)# t is an array with values ranging from 0 to t_stop with increment 0.1

  plt.subplot(3,1,2)
  title=name+' Exc Spike Trains'
  plot_spikes(excsecspikes,title)

  plt.subplot(3,1,3)
  title=name+' Inh Spike Trains'
  plot_spikes(inhsecspikes,title)
  plt.xlabel('time(ms)')
  plt.savefig('Plots/SROL5Plots/'+savename)

def subplot_voltagetrace_with1spikes(probe):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  savename=name+'.png'
  plt.figure(figsize=(15,8))
  plt.subplot(2,1,1)
  title=name+' Voltage Trace'
  plot_trace(segment,color,title,data=v_dend[segment])
  t = np.arange(0,tstop,0.1)# t is an array with values ranging from 0 to t_stop with increment 0.1

  plt.subplot(2,1,2)
  if inhsecspikes==[]:
    title=name+' Exc Spike Trains'
    plot_spikes(excsecspikes,title)
  else:
    title=name+' Inh Spike Trains'
    plot_spikes(inhsecspikes,title)
  plt.xlabel('time(ms)')
  plt.savefig('Plots/SROL5Plots/'+savename)

#plot spikes and voltage in same plot?
# def plot_voltagetrace_withspikes(probe):
#   #work in progress: plotting spikes and voltage trace in the same plot
#   plot_voltage_trace(probe)
#   nTrace = len(probe.excspikes)
#   t = np.arange(0,tstop,0.1)
#     # t is an array with values ranging from 0 to t_stop with increment 0.1
#   z = np.zeros((nTrace,t.shape[0]))
#     # Z is a nTrace by t.shape[0] matrix of 0's
#   for i in np.arange(0,z.shape[0]):
#       plt.plot(probe.excspikes[i],np.ones((probe.excspikes[i].shape[0]))*i,'k.',color='r')
#     # figure out how to make a heatmap with spikes
#     # ax = plt.scatter(AllSegXCoord, AllSegYCoord, c = np.log(ElecDist) )
#     # plt.vlines(110,300,400)
#     # plt.text(0,350,'100 um')
#     # plt.hlines(300,110,210)
#     # plt.text(110,250,'100 um')
#     # plt.xticks([])
#     # plt.yticks([])
#     # plt.title(title)
#     # cbar = plt.colorbar()
#     # cbar.ax.set_ylabel('log(elec_distance)', rotation=270)
#     # plt.box(False)


def plot_current_trace(probe,title,savename,data):
  plt.figure(figsize=(15,4))
  plt.plot(np.arange(0,((h.tstop)+.1),.1),data[probe.seg],color=probe.color)
  plt.hlines(0,0,tstop, color = 'grey', linestyle = 'dashed')
  plt.title(title)
  plt.box(False)
  plt.xticks([])
  plt.yticks([])
  plt.savefig('Plots/SROL5Plots/'+savename)

def plot_iCaLVA(probe):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  title=name+' iCaLVA'
  savename=name+'_iCaLVa.png'
  plot_current_trace(segment,color,title,savename,data=ical_data)

def plot_iH(probe):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  title=name+' iH'
  savename=name+'_iH.png'
  plot_current_trace(segment,color,title,savename,data=ih_data)

def plot_iNMDA(probe):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  title=name+' iNMDA'
  savename=name+'_iNMDA.png'
  plot_current_trace(segment,color,title,savename,data=i_NMDA_bySeg)

def plot_iCaHVA(probe):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  title=name+' iCaHVA'
  savename=name+'_iCaHVa.png'
  plot_current_trace(probe=probe,title=title,savename=savename,data=icah_data)

def plot_axial_current(probe):
  name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  axial_current=probe.axial_current
  title=name+' Axial Current'
  dend_types = axial_current[0].dend_type
  plt.figure(figsize=(12.8, 4.8))
  for i,AC in enumerate(axial_current):
      ac = AC.get_current()
      for dend_type in dend_types:
          plt.plot(t,ac[dend_type].ravel(),label=dend_type)
      plt.ylabel('nA')
      plt.legend()
      plt.title(title)
      plt.xlabel('time (ms)')
  plt.show()

def overlay_axial_current(probes):
  plt.figure(figsize=(12.8, 4.8))
  for probe in probes:
    name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
    axial_current=probe.axial_current
    title=name+' Axial Current'
    dend_types = axial_current[0].dend_type
    plt.figure(figsize=(12.8, 4.8))
    for i,AC in enumerate(axial_current):
        ac = AC.get_current()
        for dend_type in dend_types:
            plt.plot(t,ac[dend_type].ravel(),label=dend_type)
        plt.ylabel('nA')
        plt.legend()
        plt.title(title)
        plt.xlabel('time (ms)')
  plt.show()

def overlay_voltage_trace_last500ms(probe,title=None,savename=None,color=None):
  if color==None:
    name,segment,color,excsecspikes,inhsecspikes=get_probe(probe)
  else:
    name,segment,notcolor,excsecspikes,inhsecspikes=get_probe(probe)
  duration=500
  if savename==None:
    savename=name+'_Voltage.png'
  if title==None:
    title=name+' Voltage'
  v=v_dend[segment]
  plt.plot(np.arange(h.tstop-duration,(h.tstop+h.dt/2),h.dt),v[int((h.tstop-duration)/h.dt)::],color=color)
  plt.hlines(-60,tstop-duration,tstop, color = 'grey', linestyle = 'dashed')
  plt.hlines(-40,tstop-duration,tstop, color = 'grey', linestyle = 'dashed')
  plt.hlines(0,tstop-duration,tstop, color = 'grey', linestyle = 'dashed')
  plt.text(2+tstop-duration,3,'0 mV')
  plt.hlines(-10,50+tstop-duration,150+tstop-duration)
  plt.vlines(50+tstop-duration,-10,0)
  plt.text(50+tstop-duration,-15,'100 ms')
  plt.text(2+tstop-duration,-5,'10 mV')
  plt.text(2+tstop-duration,-37,'-40 mV')
  plt.text(2+tstop-duration,-57,'-60 mV')
  plt.title(title)
  plt.box(False)
  plt.xticks([])
  plt.yticks([])
  plt.savefig('Plots/SROL5Plots/'+savename)

In [ ]:
# disttuftprobes=[]
# for i in disttufts:
#   disttuftprobes.append(probes[i])
# overlay_axial_current(disttuftprobes)

In [ ]:
# dir(probes[0].axial_current[0])

In [ ]:
# %cd Plots

In [ ]:
# plot_elec_dist(AllSegBElecDist,'Beta Electrotonic Distance','Beta_Elec_distance.svg')

In [ ]:
# plot_elec_dist(AllSegGElecDist,'Gamma Electrotonic Distance','Gamma_Elec_distance.svg')

In [ ]:
plotSynDensity(excSynPerSeg)

In [ ]:
plotSynDensityNoSoma(inhSynPerSeg) #soma has too many inh synapses that the heatmap is not useful

In [ ]:
plt.figure(figsize=(3,10))
ax = plt.plot(AllSegXCoord, AllSegYCoord,'.',color='k',markersize=6)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
plt.box(False)

for i in range(len(probes)):
  plt.plot(AllSegXCoord[probes[i].seg],
         AllSegYCoord[probes[i].seg],
         '*',color=probes[i].color)
plt.savefig('Plots/Probe Locations.png')

In [ ]:
#for plotting morphology
seccolors = ['g',       #soma
'yellow','yellow','yellow','yellow',  #proxbasal
'orange','orange','orange','orange',  #midbasal
'red','red','red','red',     #distbasal
'yellow',  #proxtrunk
'purple','purple','purple','purple',  #oblique
'blue',    #midtrunk
'green',   #disttrunk
'yellow','yellow','yellow','yellow',  #proxtuft
'orange','orange','orange','orange',  #midtuft
'red','red','red','red',     #disttuft
'c',       #axon
'black','black','black','black']  #passivebasal
# 'black' #passivetuft

In [ ]:
soma=[0]
proxbasal=range(1,5)
midbasal=range(5,9)
distbasal=range(9,13)
proxtrunk=[13]
obliques=range(14,18)
midtrunk=[18]
disttrunk=[19]
proxtufts=range(20,24)
midtufts=range(24,28)
disttufts=range(28,32)
axon=[32]
passivebasal=range(33,37)

# for i in range(len(sim.cells.all)): #assemble seccolors array
#   if i in soma:
#     seccolors[i]='g'
#   elif i in proxbasal:

#corresponds to section indices in this list.
#print(sec_list)

In [ ]:
# dir(probes[0].axial_current)

In [ ]:
# len(probes[10].axial_current)

In [ ]:
# dir(probes[0])

In [ ]:
# for i in range(len(probes)):
#   print(probes[i].sectionname)

Axial Current Plots

In [ ]:
# for i in range(len(probes)):
#   if probes[i].sectionname=='proxtuft':
#     plot_axial_current(probes[i])

In [ ]:
# #Plot All
# for i in range(len(probes)):
#   # import pdb; pdb.set_trace()
#   plot_axial_current(probes[i])

In [ ]:
print(len(seccolors))

In [ ]:
for i in range(len(sim.cells[0].all)-len(seccolors)):
  seccolors.append('b')

In [ ]:
plot_morphology(sim,electrodes=elec_pos,elev=-10, azim=20, figsize=(10,12), clr = seccolors)
plt.savefig('Morphology.png')

Voltage Traces

In [ ]:
# #sfn plot
# plot_voltage_trace_last500ms(probes[0],title='Soma Voltage',savename='SomaVoltage.png',color='black')
# # plot_voltage_trace_last500ms(probes[disttrunk[0]],title='Nexus Voltage',savename='NexusVoltage.png')
# plot_voltage_trace_last500ms(probes[proxtufts[0]],title='Proximal Tuft Dendrite Voltage',savename='ProxTuftVoltage.png',color='green')
# plot_voltage_trace_last500ms(probes[obliques[0]],title='Oblique Dendrite Voltage',savename='ObliqueVoltage.png',color='blue')
# plot_voltage_trace_last500ms(probes[distbasal[0]],title='Distal Basal Dendrite Voltage',savename='DistBasalVoltage.png',color='orange')
# plot_voltage_trace_last500ms(probes[midbasal[0]],title='Middle Basal Dednrite Voltage',savename='MidBasalVoltage.png',color='grey')
# plot_voltage_trace_last500ms(probes[proxtrunk[0]],title='Proximal Trunk Voltage',savename='ProxTrunkVoltage.png',color='purple')
# # plot_voltage_trace_last500ms(probes[midtufts[0]],title='Middle Tuft Dendrite Voltage',savename='MidTuftVoltage.png')
# plot_voltage_trace_last500ms(probes[disttufts[0]],title='Distal Tuft Dendrite Voltage',savename='DistTuftVoltage.png',color='red')
colorlist=['black','green','blue','orange','grey','purple','red']
probeslist=[probes[0],probes[proxtufts[0]],probes[obliques[0]],probes[distbasal[0]],probes[midbasal[0]],probes[proxtrunk[0]],probes[disttufts[0]]]
plt.figure(figsize=(3,10))
ax = plt.plot(AllSegXCoord, AllSegYCoord,'.',color='k',markersize=6)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
plt.box(False)

for i in range(len(probeslist)):
  plt.plot(AllSegXCoord[probeslist[i].seg],
         AllSegYCoord[probeslist[i].seg],
         '*',color=colorlist[i])
plt.savefig('ProbeLocations.png')

In [ ]:
plot_iCaHVA(probes[midtufts[0]])

In [ ]:
plot_voltage_trace_last500ms(probes[0],title='Soma Voltage',savename='SomaVoltage.png',color='black')
# plot_voltage_trace_last500ms(probes[disttrunk[0]],title='Nexus Voltage',savename='NexusVoltage.png')
plot_voltage_trace_last500ms(probes[proxtufts[0]],title='Proximal Tuft Dendrite Voltage',savename='ProxTuftVoltage.png',color='green')
plot_voltage_trace_last500ms(probes[obliques[0]],title='Oblique Dendrite Voltage',savename='ObliqueVoltage.png',color='blue')
plot_voltage_trace_last500ms(probes[distbasal[0]],title='Distal Basal Dendrite Voltage',savename='DistBasalVoltage.png',color='orange')
plot_voltage_trace_last500ms(probes[midbasal[0]],title='Middle Basal Dednrite Voltage',savename='MidBasalVoltage.png',color='grey')
plot_voltage_trace_last500ms(probes[proxtrunk[0]],title='Proximal Trunk Voltage',savename='ProxTrunkVoltage.png',color='purple')
# plot_voltage_trace_last500ms(probes[midtufts[0]],title='Middle Tuft Dendrite Voltage',savename='MidTuftVoltage.png')
plot_voltage_trace_last500ms(probes[disttufts[0]],title='Distal Tuft Dendrite Voltage',savename='DistTuftVoltage.png',color='red')

In [ ]:
# for i in soma:
#   subplot_voltagetrace_with1spikes(probes[i])

In [ ]:
# for i in proxbasal:
#   subplot_voltagetrace_withspikes(probes[i])
# for i in midbasal:
#   subplot_voltagetrace_withspikes(probes[i])
# for i in distbasal:
#   subplot_voltagetrace_withspikes(probes[i])

In [ ]:
# for i in disttrunk:
#   plot_voltage_trace(probes[i])

In [ ]:
# for i in proxtufts:
#   subplot_voltagetrace_withspikes(probes[i])
# for i in midtufts:
#   subplot_voltagetrace_withspikes(probes[i])
# for i in disttufts:
#   subplot_voltagetrace_withspikes(probes[i])

In [ ]:
# for i in proxtrunk:
#   subplot_voltagetrace_with1spikes(probes[i])

In [ ]:
# for i in obliques:
#   subplot_voltagetrace_with1spikes(probes[i])

In [ ]:
# for i in midtrunk:
#   subplot_voltagetrace_with1spikes(probes[i])

In [ ]:
# for i in axon:
#   plot_voltage_trace(probes[i])
# for i in passivebasal:
#   plot_voltage_trace(probes[i])

In [ ]:
#plot all traces and spike trains

# for i in range(len(probes)):
#   subplot_voltagetrace_withspikes(probes[i])

In [ ]:
# %cd ..

##Current Traces for every segment

In [ ]:
def plot_trace(data, title, ylabel):
  plt.figure(figsize=(20,14))
  for i in range(data.shape[0]):
      plt.plot(t,data[i,:])
      plt.title(title, loc='center')
      plt.ylabel(ylabel)
      plt.xlabel('ms')
  plt.show()

In [ ]:
# # Ca LVA current 
# plot_trace(data=ical_data, title='ical', ylabel='ical')

In [ ]:
sec14 = []
for i in range(nseg-1):
  if AllSegSection[i] == 12:
    sec14.append(i)

# len(sec14)

In [ ]:
# # CA HVA current
# plot_trace(data=icah_data,title='ICaH',ylabel='icah')

In [ ]:
# # h current
# plot_trace(data=ih_data,title='ih',ylabel='Ih')

In [ ]:
sec = []
for i in range(nseg-1):
  if AllSegSection[i] >= 11 :
    sec.append(i)

print(len(sec))

# plt.figure(figsize=(20,14))
# for i in sec:
#     plt.plot(t,gNaTa_T_data[i,:])
#     plt.ylabel('gNaTa_T')
#     plt.xlabel('ms')
#     plt.title('gNaTa_T')
# plt.show()

# plt.figure(figsize=(20,14))
# for i in sec14:
#     plt.plot(t,gNaTa_T_data[i,:])
#     plt.title(gNaTa_T, loc='center')
#     plt.ylabel('gNaTa_T')
#     plt.xlabel('ms')
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline


# #NMDA current 

# plt.figure(figsize=(20,14))
# for i in range(len(i_NMDA_bySeg)):
#     plt.plot(t,i_NMDA_bySeg[i])
#     plt.ylabel('i_NMDA')
#     plt.xlabel('ms')
#     plt.title('i_NMDA in every segment')
# plt.show()

In [ ]:
# Voltage traces

# plt.figure(figsize=(20,14))
# for i in range(len(v_dend)):
#     plt.plot(t,v_dend[i])
#     plt.ylabel('v dend ')
#     plt.xlabel('ms')
#     plt.title('All Voltage Traces')
# plt.show()

# Passive Properties

In [ ]:
# tstop=4000
# h.tstop = tstop

In [ ]:
# timestart=time.time()
# PassiveSim = Simulation(geo_standard,elec_pos,loc_param,geo_param=geo_param,biophys=biophys,gmax=0.05,scale=100.,spike_threshold = 10, RunType = 'Passive Properties', synaptic_input=False)  # 0.001 -0.012 #gmax=.005, scale=100
# timestop=time.time()
# elapseddeftime=timestop-timestart
# simtime=tstop/1000 #convert from ms to s
# print('It took',round(elapseddeftime),'sec to define the simulation model.') #need to optomize input grouping # location check takes a long time since it iterates through checking a list of synapse locations for each group/cluster within the section

In [ ]:
# dend_v = Recorder(Vrecord)

In [ ]:
# timestart=time.time()
# PassiveSim.run()
# timestop=time.time()
# lfp = sim.get_lfp().T
# t = PassiveSim.t()
# elapsedtime=timestop-timestart
# simtime=tstop/1000 #convert from ms to s
# totaltime= elapsedtime+elapseddeftime
# print('It took',round(elapsedtime),'sec to run a',simtime,'sec Passive Properties simulation.')
# print('The total runtime was',round(totaltime),'sec')

In [ ]:
# v_dend = dend_v.as_numpy()

In [ ]:
# print(len(PassiveSim.cells[0].injection))

In [ ]:
# spikes = PassiveSim.get_spike_time()

# print(len(spikes))

# HzAt100Pa = len(spikes[spikes < 1000])*1.3333
# HzAt200Pa = len(spikes[(spikes > 1000) & (spikes < 2000)])*1.3333
# HzAt300Pa = len(spikes[(spikes > 2000) & (spikes < 3000)])*1.3333
# HzAt400Pa = len(spikes[spikes > 3000])*1.3333

# pA = [100,200,300,400]
# Hz = [HzAt100Pa,HzAt200Pa,HzAt300Pa,HzAt400Pa]


# plt.figure(figsize=(7,8))
# ax = plt.plot(pA,Hz)
# plt.title('New model F-I curve')
# plt.xlabel('Current Injection (pA)')
# plt.ylabel('Firing Rate (Hz)')
# plt.savefig('FI_curve.png')
# plt.show()

In [ ]:
# plot soma voltage
# plot_voltage_trace(segment=soma_probe,color='grey',title='Soma Voltage',savename='Passive_Properties_Soma_Voltage.png')

# Active Properties

In [ ]:
# tstop=1500
# h.tstop = tstop

In [ ]:
# timestart=time.time()
# ActiveSim = Simulation(geo_standard,elec_pos,loc_param,geo_param=geo_param,biophys=biophys,gmax=0.05,scale=100.,spike_threshold = 10, RunType = 'Active Properties', synaptic_input=False)  # 0.001 -0.012 #gmax=.005, scale=100
# timestop=time.time()
# elapseddeftime=timestop-timestart
# simtime=tstop/1000 #convert from ms to s
# print('It took',round(elapseddeftime),'sec to define the simulation model.') #need to optomize input grouping # location check takes a long time since it iterates through checking a list of synapse locations for each group/cluster within the section

In [ ]:
# dend_v = Recorder(Vrecord)

In [ ]:
# timestart=time.time()
# ActiveSim.run()
# timestop=time.time()
# lfp = sim.get_lfp().T
# t = sim.t()
# elapsedtime=timestop-timestart
# simtime=tstop/1000 #convert from ms to s
# totaltime= elapsedtime+elapseddeftime
# print('It took',round(elapsedtime),'sec to run a',simtime,'sec simulation.')
# print('The total runtime was',round(totaltime),'sec')

In [ ]:
# v_dend = dend_v.as_numpy()

In [ ]:
# from pandas.core.internals.managers import create_block_manager_from_arrays
# v_rest = v_dend[0][1000]
# print('v_rest')
# print(v_rest)
# v_final = v_dend[0][5000]
# print('v_final')
# print(v_final)

# calc = v_rest -((v_rest - v_final) * 0.638)

# print('calc')
# print(calc)


# i = 150 
# while v_dend[0][i] > calc:
#   i = i + 1

# print('v at tic')
# print(v_dend[0][i])


# timeAtV = i/10
# print('timeAtV', timeAtV)
# print('i: ', i)
# t = timeAtV - 150
# t_ins = t/1000

# print('t')
# print(t)
# print('t_ins')
# print(t_ins)


# deltaV = (v_rest - v_final)
# deltaI = (0 - (-100))
# dVdI = deltaV / (deltaI /1000)

# print('dVdI')
# print(dVdI)

In [ ]:
# print(len(ActiveSim.cells[0].injection))

In [ ]:
#plot soma voltage
# plot_voltage_trace(segment=soma_probe,color='grey',title='Soma Voltage',savename='Active_Properties_Soma_Voltage.png')